This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_best_worst_per_point(y_true, y_pred_list, savepath, metrics_dict,
                              avg_stats, title='best worst per point', label='target_value'):
    """
    Method to create a parity plot (predicted vs. true values) of the set of best and worst CV scores for each

    individual data point.

    Args:

        y_true: (numpy array), array of true y data

        y_pred_list: (list), list of numpy arrays containing predicted y data for each CV split

        savepath: (str), path to save plots to

        metrics_dict: (dict), dict of scikit-learn metric objects to calculate score of predicted vs. true values

        avg_stats: (dict), dict of calculated average metrics over all CV splits

        title: (str), title of the best_worst_per_point plot

        label: (str), label used for axis labeling

    Returns:

        None

    """

    worsts = []
    bests = []
    new_y_true = []
    for yt, y_pred in zip(y_true, y_pred_list):
        if len(y_pred) == 0 or np.nan in y_pred_list or yt == np.nan:
            continue
        worsts.append(max(y_pred, key=lambda yp: abs(yp-yt)))
        bests.append( min(y_pred, key=lambda yp: abs(yp-yt)))
        new_y_true.append(yt)

    worst_stats = OrderedDict([('Worst combined:', None)])
    best_stats = OrderedDict([('Best combined:', None)])
    for name, (_, func) in metrics_dict.items():
        worst_stats[name] = func(new_y_true, worsts)
        best_stats[name] = func(new_y_true, bests)

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 15.5/24 #mmm yum
    fig, ax = make_fig_ax(x_align=x_align)

    # gather max and min
    #all_vals = [val for val in worsts+bests if val is not None]
    max1 = max(y_true)
    min1 = min(y_true)

    # draw dashed horizontal line
    ax.plot([min1, max1], [min1, max1], 'k--', lw=2, zorder=1)

    # set axis labels
    ax.set_xlabel('True '+label, fontsize=16)
    ax.set_ylabel('Predicted '+label, fontsize=16)

    # set tick labels
    #maxx = max((max(bests), max(worsts), max(new_y_true)))
    #minn = min((min(bests), min(worsts), min(new_y_true)))
    #maxx, minn = recursive_max_and_min([bests, worsts, new_y_true])
    maxx = round(float(max1), rounder(max1-min1))
    minn = round(float(min1), rounder(max1-min1))
    _set_tick_labels(ax, maxx, minn)

    ax.scatter(new_y_true, bests,  c='red',  alpha=0.7, label='best test',
               edgecolor='darkred',  zorder=2, s=100)
    ax.scatter(new_y_true, worsts, c='blue', alpha=0.7, label='worst test',
               edgecolor='darkblue', zorder=3, s=60)
    ax.legend(loc='lower right', fontsize=12)

    plot_stats(fig, avg_stats, x_align=x_align, y_align=0.51, fontsize=10)
    plot_stats(fig, worst_stats, x_align=x_align, y_align=0.73, fontsize=10)
    plot_stats(fig, best_stats, x_align=x_align, y_align=0.95, fontsize=10)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([177.28302038, 150.37102595, 202.0924573 , ...,  47.20790294,
        23.809066  ,   0.        ])
y_pred_list = [[0.0, 2.4682426846407353e-05], [2.371624071596713e-05, 2.3658847472516923e-05], [2.7270187136234468e-05, 0.0], [4.023137020657062e-05, 4.023137020657062e-05], [0.0, 1.7534702229765136e-05], [3.020681691916944e-143, 3.020681691916944e-143], [8.654742247191032e-236, 8.654742247191032e-236], [3.1768835408671962e-143, 1.1107614030342804e-248], [0.0, 0.0], [9.375581443006237e-249, 0.0], [1.4872012724385833e-05, 1.4872012724385833e-05], [1.5767041180087988e-122, 3.229139508277829e-143], [1.25837470535106e-270, 1.25837470535106e-270], [1.7326186548588933e-122, 1.7326186548588933e-122], [0.0, 0.0], [4.9251665514999466e-73, 4.9251665514999466e-73], [1.9952036884816004e-05, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [3.216337106523557e-05, 3.216337106523557e-05], [9.128905378878565e-123, 8.078081943050533e-249], [5.248415899805766e-236, 5.248415899805766e-236], [9.267838920285754e-123, 9.267838920285754e-123], [0.0, 0.0], [5.949497913213695e-249, 0.0], [2.097317937173078e-304, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [4.569101125501751e-28, 4.569101125501751e-28], [4.5434410662890515e-28, 4.5434410662890515e-28], [3.5625866961788785e-28, 3.5625866961788785e-28], [9.120640855669859e-28, 9.120640855669859e-28], [2.780149168454274e-28, 9.626135147853923e-28], [8.782225599264033e-135, 8.782225599264033e-135], [1.6136553760741216e-274, 6.232921137233406e-192], [3.2801877579940577e-133, 3.2801877579940577e-133], [2.478347338148707e-231, 2.478347338148707e-231], [5.311578336550062e-192, 5.311578336550062e-192], [9.64563317604946e-28, 9.64563317604946e-28], [8.225587987025556e-135, 1.251313001704459e-132], [6.1339549948314385e-192, 2.0910710715972548e-274], [1.938412329733536e-132, 1.938412329733536e-132], [2.060428348222292e-231, 2.060428348222292e-231], [5.6075717978876164e-192, 0.0], [2.4397635955296342e-28, 7.940811891610864e-28], [0.0, 4.423028451384998e-192], [0.0, 4.249892511058777e-192], [3.979939546432061e-192, 3.979939546432061e-192], [2.166690304828953e-231, 2.166690304828953e-231], [0.0, 3.866026856851203e-192], [7.090974638373479e-28, 7.090974638373479e-28], [8.065011358421963e-192, 9.080955181726702e-133], [6.901906666283866e-192, 2.365970890380677e-238], [8.991410211332597e-133, 8.991410211332597e-133], [4.3033559323208156e-231, 4.3033559323208156e-231], [0.0, 7.944974502898434e-192], [1.1913199653915117e-115, 4.5675919416414e-28], [4.540355825541815e-28, 4.540355825541815e-28], [1.0450487040188975e-115, 3.564935466953138e-28], [3.035922536396033e-115, 3.035922536396033e-115], [1.895070350121882e-27, 1.895070350121882e-27], [1.9894007111597417e-27, 1.9894007111597417e-27], [1.6812030704924218e-27, 1.6812030704924218e-27], [3.9475277014487565e-27, 3.9475277014487565e-27], [1.2107030459243812e-27, 1.2107030459243812e-27], [7.13326363399317e-135, 7.13326363399317e-135], [2.706397130022194e-238, 5.964360374128644e-192], [5.247536251784122e-133, 5.247536251784122e-133], [3.3987191629241314e-231, 3.3987191629241314e-231], [4.898405567791188e-192, 4.898405567791188e-192], [1.3130482429012406e-27, 1.3130482429012406e-27], [1.0505615532475022e-132, 1.0505615532475022e-132], [4.593974776638921e-192, 2.1894185808807105e-274], [1.758772846224857e-132, 3.5806750084612894e-133], [2.4069831196627033e-231, 2.4069831196627033e-231], [5.154776886436552e-192, 0.0], [1.1399484441845894e-27, 1.1399484441845894e-27], [0.0, 5.867430217388495e-192], [0.0, 4.454173162627695e-192], [4.350934544400233e-192, 4.350934544400233e-192], [2.332798565046906e-231, 2.332798565046906e-231], [0.0, 4.120076529116229e-192], [3.020899600963595e-27, 3.020899600963595e-27], [7.313959459820532e-133, 7.313959449409723e-133], [7.513327917873944e-192, 6.870058979002043e-281], [1.4458413613370927e-132, 8.254564022560971e-192], [4.7890213109508315e-231, 4.7890213109508315e-231], [1.1585386671528398e-239, 8.421615172099054e-192], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [5.220413782254054e-308, 5.220413778538798e-308], [1.9301747188293185e-10, 1.9301747188293185e-10], [2.0203112986101183e-10, 2.0203112986101183e-10], [0.0, 0.0], [3.727034351210053e-10, 3.727034351210053e-10], [1.4550286688738512e-10, 1.4550286688738512e-10], [2.255161025292685e-130, 2.255161025292685e-130], [7.7055863258085505e-236, 0.0], [2.8709730808574144e-130, 2.8709730808574144e-130], [8.399587549851054e-236, 0.0], [8.039098034095712e-236, 8.039098034095712e-236], [1.4717189189508623e-10, 1.4717189189508623e-10], [2.185080453737937e-130, 1.324377833333262e-122], [1.1003685832601519e-270, 0.0], [1.5466010408321754e-122, 2.744970732700965e-130], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [3.243389567592736e-10, 3.243389567592736e-10], [5.760333022742272e-236, 7.41972820741936e-123], [4.188637714938937e-236, 0.0], [9.120261378862805e-123, 5.145998166626833e-236], [4.8852718790342616e-236, 0.0], [4.789190606194714e-236, 4.789190609603083e-236], [3.0738195218938786e-304, 3.0738195218938786e-304], [0.0, 0.0], [0.0, 0.0], [1.8528231254233805e-304, 1.8528231254233805e-304], [2.971059626987417e-32, 2.971059626987417e-32], [2.3252238473649087e-32, 2.3252238473649087e-32], [0.0, 1.9188370236972293e-32], [5.694206786222412e-32, 5.694206786222412e-32], [2.1022462595472387e-32, 2.1022462595472387e-32], [8.420273927466924e-148, 8.420273927466924e-148], [2.7152199539954525e-238, 3.059962385476358e-274], [3.776094726920481e-133, 3.7760947215455265e-133], [1.7723864785562484e-252, 1.7723864785562484e-252], [1.9068515423055418e-252, 1.9068515423055418e-252], [1.368200421061995e-32, 1.368200421061995e-32], [1.0796723683149269e-132, 1.0796723683149269e-132], [1.744536381295857e-274, 1.7445363850207068e-274], [1.59735845983663e-132, 1.59735845983663e-132], [0.0, 0.0], [0.0, 0.0], [0.0, 1.4863343921627753e-32], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [4.619970340057406e-32, 4.619970336769401e-32], [7.919999268924848e-133, 7.919999268924848e-133], [1.959644807173974e-238, 9.865782784203789e-281], [1.0249728060273828e-132, 1.0249728074863456e-132], [1.0551729822939427e-252, 1.0551729822939427e-252], [1.1886413644337787e-252, 1.1886413644337787e-252], [1.0415979486061481e-305, 0.0], [0.0, 0.0], [0.0, 0.0], [6.01025560603656e-306, 0.0], [1.59223041841346e-36, 1.59223041841346e-36], [1.0539104561507505e-08, 1.0539104561507505e-08], [1.6174193850466334e-08, 1.6174193850466334e-08], [7.669269724519893e-258, 7.669269724519893e-258], [6.454396526742157e-258, 2.951203094760154e-242], [3.110942742270056e-119, 7.27068787280893e-269], [1.1395943659012859e-119, 5.757542080477268e-29], [6.571123974119706e-255, 3.102656958403847e-29], [6.182563099673187e-13, 6.182563099673187e-13], [1.9745068200831803e-12, 1.9745068200831803e-12], [0.006580453855934725, 0.006580453855934725], [31.681367392932753, 31.681367392932753], [4.2048091715688955e-14, 4.2048091715688955e-14], [6.93296219243089e-14, 6.93296219243089e-14], [2.2814991024604268e-156, 2.2814991024604268e-156], [118.34105450711688, 118.34105450711688], [4.271526550623338e-38, 4.271526550623338e-38], [1.5315361627860725e-157, 1.208492184596392e-157], [1.005987806643546e-38, 1.005987806643546e-38], [1.5964865100869708e-38, 1.5964865100869708e-38], [0.0, 0.0], [0.3415003726938544, 0.3415003726938544], [0.2825047796437008, 0.28250477964370085], [6.556687213958943e-08, 1.608869773306009e-07], [8.990695717972111e-35, 9.064189501851904e-08], [2.04415912783464e-23, 2.450525431747048e-21], [1.849240015768577e-23, 1.7649956885337518e-21], [9.207671168556272e-137, 9.207671168556272e-137], [5.1264142011164324e-27, 5.1264142011164324e-27], [4.018744722567056e-27, 4.018744728287345e-27], [9.927290100064399e-111, 9.927290100064399e-111], [5.987873335567473e-111, 5.987873335567473e-111], [0.0, 0.0], [3.7136961571200442e-25, 3.7136961571200442e-25], [3.1636522813137133e-25, 3.1636522813137133e-25], [1.593683458949572e-103, 2.7391888338761725e-103], [0.0, 1.7538743845317176e-103], [8.605380488432997e-137, 8.605380488432997e-137], [3.9360917893599525e-28, 1.6153478294076087e-132], [3.0197361394200105e-28, 1.7503728758899795e-115], [2.8921346210825833e-115, 2.8921346210825833e-115], [1.7480191264605165e-115, 1.7480191264605165e-115], [1.6173831530230468e-20, 1.6173831530230468e-20], [2.3540208315333135, 2.3540208315333127], [2.087287402554899, 2.0875963855623763], [6.970948686375435e-11, 0.0002634365911021853], [1.7706602455029454e-20, 1.7706602455029454e-20], [2.176887860471119, 2.1768878604828736], [1.7719533419818219, 1.7719533419818219], [0.0002439863382201867, 0.00024398633822020495], [3.981089939736619e-62, 3.9810897517305204e-62], [0.0, 0.0], [0.0010819252203019892, 0.0010819252203019892], [0.0007142125451948744, 0.0007142125451948744], [3.4898013820758925e-44, 1.802712800473222e-18], [8.331537519837117e-125, 1.4276463613772838e-18], [1.6325779039187137e-183, 1.6325779039187137e-183], [3.669212145139701e-22, 3.669212145139701e-22], [3.5685691631470526e-22, 3.5685691631470526e-22], [1.001002665351376e-90, 1.001002665351376e-90], [5.925612686209313e-91, 5.925612686209313e-91], [2.1349288004722647e-21, 2.134928800472265e-21], [2.1904038796218242e-21, 2.1572929976832398e-21], [0.0, 0.0], [1.673034561376003e-22, 1.2906760424366467e-209], [1.4937273845516644e-22, 1.635503428188486e-92], [1.846826065473758e-92, 9.1374782194657e-93], [1.6342022955163116e-92, 0.0], [0.0, 0.0], [2.6437447328224337, 2.6437447329922774], [2.535701839876991, 2.5349823915770355], [0.0013313719447073467, 6.128600082110677e-10], [0.0007194805048668034, 3.582346616745069e-29], [0.0, 0.0], [1.843753458857716e-10, 1.8437534601699107e-10], [1.5302201317650057e-10, 1.530220130675952e-10], [5.403558998583441e-45, 9.034326051302733e-45], [4.6956971006396605e-45, 4.6956971006396605e-45], [61.155677834999985, 61.155677834999985], [0.0, 0.0], [2.5637317782000014e-06, 9.627678373919611], [0.0420480533282607, 7.492275157631975], [0.07745631927880864, 0.02705199670020237], [0.0420249123361647, 2.070808470743948e-12], [0.0, 0.0], [1.2149577334992246e-05, 1.214957730905178e-05], [9.657662918949113e-06, 9.657662918949113e-06], [2.356170125587388e-25, 2.3561701875979325e-25], [1.1746184077335095e-25, 1.1746184077335095e-25], [0.0, 0.0], [1.2784079208147572e-13, 1.2784079208147572e-13], [1.3151287960312885e-13, 1.3151287960312885e-13], [31.522867395583642, 31.522867395583642], [0.0, 0.0], [0.0, 0.0], [2.9433663975352755e-116, 2.9433663975352755e-116], [0.0, 0.0], [0.0, 0.0], [4.445335994139814e-09, 3.134869356810974e-91], [3.166311048151206e-09, 1.0456369367651807e-39], [5.395758807490222e-40, 5.395758807490222e-40], [0.0, 0.0], [1.4057384590000534e-38, 1.4057384590000534e-38], [1.7028199465199328e-38, 1.7028199465199328e-38], [0.0, 0.0], [0.0, 4.216893269564255e-118], [0.0, 4.971994356827453e-118], [0.0, 0.0], [0.0, 0.0], [1.9463740168849125e-223, 4.4415586909671276e-24], [5.6190605206659056e-99, 4.86408509967445e-24], [9.358547517621954e-99, 9.358547510961665e-99], [0.0, 0.0], [1.899817674494994e-98, 1.899817674494994e-98], [2.452158311463209e-98, 2.452158311463209e-98], [0.0, 0.0], [0.0, 0.0], [2.6316257885510945e-76, 2.6316257885510945e-76], [1.3558813638642748e-75, 1.3558813638642748e-75], [1.956047741037342e-304, 1.9560477382531835e-304], [0.0, 0.0], [2.981372228159255e-26, 2.981372228159255e-26], [4.583113873575233e-26, 4.583113873575233e-26], [1.2977447945382132e-106, 2.313168271874899e-240], [0.0, 0.0], [5.439158835486278e-27, 5.439158835486278e-27], [1.5277874720538847e-26, 1.5277874720538847e-26], [3.233193737145094e-108, 3.233193737145094e-108], [0.0, 0.0], [3.591480001637856e-39, 3.591480001637856e-39], [7.138905210141085e-39, 7.138905210141085e-39], [0.0, 3.2338328919342782e-158], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [8.21371737075234e-27, 8.21371737075234e-27], [9.59903240680893e-27, 9.59903240680893e-27], [1.4444001298531988e-109, 4.201153336083418e-142], [0.0, 0.0], [8.489160401437546e-08, 8.248697937835227e-23], [1.96357113962543e-09, 1.96356897871346e-09], [3.876948082964589e-09, 3.876948086633184e-09], [1.96058787700284e-09, 1.960587876990151e-09], [0.0, 0.0], [1.228881239186969e-10, 0.04215348521827406], [0.03148804736227572, 0.08239281914155255], [0.04212711289364304, 0.04212711289364304], [1.6405973813266177e-10, 1.6405973813266177e-10], [8.253368549671022e-11, 3.8067678780584485e-44], [-2.996821819044486e-47, -6.497342567567635e-14], [0.012838456736449818, 0.012838456736449818], [0.004423652598266465, 0.004423652598266465], [1.2056877955917643e-12, 1.2056877955917643e-12], [0.00015757763274196077, 0.0001575776326298132], [5.88984428104126e-05, 5.88984428104126e-05], [1.0402270566012503e-20, 1.0402278418439289e-20], [-2.4957343342468597e-95, -2.4957343342468597e-95], [3.249682613750088e-76, 3.249682613750088e-76], [0.0, 0.0], [2.4602400412260897e-08, 2.3262973632058093e-83], [1.012003490525574e-08, 3.0880286756307335e-36], [1.3127476088725471e-36, 1.3127476088725471e-36], [1.75979936973573e-252, 0.0], [3.339097961481647e-112, 3.339097961481647e-112], [0.0, 0.0], [2.2270516259765462e-05, 2.2270516259765462e-05], [7.623028379445256e-06, 7.623028379445256e-06], [6.942774920571551e-24, 3.3817177327590534e-55], [0.0, 0.0], [4.309295079898435e-180, 4.309295079898435e-180], [0.0, 0.0], [0.0, 0.0], [3.504524141781852e-67, 9.894008591904668e-07], [3.53193396671503e-29, 3.882193458109634e-07], [1.577428925478202e-29, 1.5774289331972898e-29], [1.5884651441852224e-108, 1.4064206880374671e-47], [1.8788247686465784e-11, 1.8788247686465784e-11], [3.5087720961286436e-11, 3.5087720961286436e-11], [3.8847455581824004e-278, 3.8847455581824004e-278], [12.445371009682702, 12.445371009682702], [0.0, 1.7790091064993816e-242], [2.6461846719785504e-131, 2.6461846719785504e-131], [5.580527221395058e-32, 5.580527221395058e-32], [4.9122840811783726e-32, 4.9122840811783726e-32], [5.304904685177708e-24, 5.304904685177708e-24], [1.466746290603324e-23, 1.466746290603324e-23], [2.0874162407878764e-54, 1.7093422310184405e-05], [0.0, 0.0], [7.646882310960657e-103, 2.0269196477610815e-10], [5.277636325467082e-45, 1.7290119443204305e-10], [9.512776279386582e-45, 9.512776279386582e-45], [5.305100072220212e-45, 5.305100072220212e-45], [0.0, 0.0], [2.8340096714849665e-06, 10.891243516844334], [0.046482197066011796, 8.643068114446919], [0.08438810166123042, 0.08438810166123042], [0.04646669828494145, 0.04646634445112428], [0.0, 0.0], [1.374696306333672e-05, 1.374696306333672e-05], [1.1200596081199773e-05, 1.1200596081199773e-05], [1.6185918732150656e-25, 2.550735890963384e-25], [1.3030527721330224e-25, 1.3030527721330224e-25], [0.0, 0.0], [0.1588849292795688, 0.1588849292795688], [0.12647104112052654, 0.12647104112052654], [4.428330044436715e-09, 4.428330044436715e-09], [2.1821304358506107e-09, 2.1821304358357384e-09], [4.853547820410607e-84, 4.853547820410607e-84], [2.4601782318884994e-84, 2.460178233639358e-84], [5.7291476240785965e-95, 1.352515e-317], [3.813446299416967e-95, 3.613814098495784e-234], [2.6712111999957067e-10, 2.6712111999957067e-10], [2.2457052421288488e-10, 2.2457052421288488e-10], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [5.8579343264034885e-56, 5.8579343264034885e-56], [0.7076122051208132, 0.7076122051208132], [0.5829753676244621, 0.5829753676244621], [1.6915675921517386e-236, 3.375257337404339e-06], [3.414887520616072e-234, 2.4686758388017395e-06], [3.3257507089338146e-72, 3.3257507089338146e-72], [2.437418933194397e-72, 2.437418933194397e-72], [6.712525707591137e-237, 0.21186804943788942], [2.134093255028339e-238, 0.12147126590839848], [1.4323844072830914e-12, 0.9541628506220605], [5.32517744570551e-06, 0.992572460500649], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.05762201732548746, 0.05762201732548746], [0.05385782883549779, 0.05385782883549779], [0.09955165461079604, 0.09955165461079604], [0.10356057814879757, 0.10356056909756653], [7.621539908394944e-09, 1.3993368548607101e-08], [0.0, 5.447767872303922e-249], [2.5689181860673177e-84, 2.5689181860673177e-84], [6.887756453579023e-85, 6.887756453579023e-85], [6.558522534425808e-241, 6.558522529758242e-241], [4.476011379689029e-238, 4.476011379689029e-238], [8.994539288226921e-239, 8.994539288226921e-239], [0.0, 0.0], [0.0, 0.0], [-6.97627037583661e-250, -7.045051318781659e-250], [9.92751472983568e-239, 0.0], [-4.841590292697016e-241, 1.1946456118835183e-240], [-7.188706615228386e-239, -7.188706618200538e-239], [0.0, 0.0], [0.0, 0.0], [5.741518219893968e-241, 3.323038384035525e-73], [4.730362842583023e-238, 4.730362842583023e-238], [3.2049070868870687e-19, 3.211355434899055e-19], [-9.099967695947083e-17, -9.091459027811208e-28], [2.201512908703365e-240, 7.381201493984912e-08], [2.3584855583249684e-31, 2.3584855583249684e-31], [-4.44094254246716e-298, 8.336434491075926e-271], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [6.58523340702751e-31, -3.86900073006268e-58], [-3.642028303459801e-27, -1.9516114730432338e-44], [4.258189438394219e-13, 4.258189438394219e-13], [-1.480902894522939e-105, 1.2635402776431942e-269], [-4.9457082478060966e-76, -1.3819479319986237e-46], [1.1505105654233598e-22, 1.1505105654233598e-22], [0.0, 0.0], [1.4898756162024282e-85, 1.5306824157621621e-238], [2.486722034028194e-84, 2.486722034028194e-84], [4.5227112178504e-97, 4.5227112178504e-97], [1.6224221551584384e-236, 1.6224221551584384e-236], [7.105365000346178e-239, 7.105365000346179e-239], [9.207637708715374e-237, 2.2324778405581776e-236], [0.0, 0.0], [0.0, 0.0], [0.0, 6.971777137646402e-239], [1.4277433310334261e-236, 1.4277433310334261e-236], [-5.27401440494632e-237, 1.0157019351641334e-238], [2.6974724074613432e-235, 2.6974898522252124e-235], [7.707603376670064e-239, 7.707603376670064e-239], [1.9624794536868656e-236, 1.9624794536868656e-236], [5.897327646324529e-239, 5.897327646324529e-239], [1.3148014214454756e-236, 1.3148014214454756e-236], [-1.289141869317866e-07, -1.289070501500509e-07], [5.313292226094201e-06, 5.313586389958728e-06], [0.00010634479269231818, 0.00010903373998153065], [0.0035553328364519433, 0.0035553328364519433], [1.248812191041055e-242, 1.248812191041055e-242], [1.3199123781137062e-260, -1.0229321985748156e-256], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [2.7248123873461574, -9.228530002234294e-09], [-9.201900592151894e-09, -2.7607603158625782e-05], [0.05595863557337337, 0.05896659641555813], [8.752065049524708e-11, -1.9394190470187373e-15], [6.125243266849547e-30, -3.0357433574966146e-09], [4.0721642250568925e-10, 0.0005483578002196175], [0.0, 4.942453867884443e-251], [7.020258218404234e-84, 7.020258218404234e-84], [3.907995620899196e-84, 3.907995620899196e-84], [1.0625540971093839e-94, 1.0625540971093839e-94], [5.6397740237578125e-95, 5.6397740237578125e-95], [1.2110015619552922e-10, 1.2110015619552922e-10], [9.577391622005372e-11, 9.577391622005372e-11], [3.0343711469754365e-241, 0.00020604352372779626], [1.1472968171156123e-237, 0.00019848235305593967], [2.32323209177947e-12, 2.32323209177947e-12], [1.856758923349646e-12, 1.856758923349646e-12], [0.2650893943039467, 0.2650893943039467], [0.21685200793229817, 0.21685200793229817], [1.2767167731515964e-06, 2.543401896974748e-241], [9.894188825693454e-07, 7.298791331355133e-239], [9.961317527850002e-72, 9.961317527850002e-72], [5.555910682567879e-72, 5.555910682567879e-72], [6.178769023709659e-66, 3.536692619627813e-07], [4.365055575250754e-29, 2.528053723797774e-07], [2.7490588808327946e-29, 2.7490589227618744e-29], [1.2097474493431786e-243, 1.2097474493431788e-243], [2.8354177284590595e-07, 2.8354177284590595e-07], [8.604760449717342e-70, 2.222880720208017e-31], [-2.895907171811011e-249, 0.0], [3.316233705545316e-242, 3.3162337055453165e-242], [2.657590880891089e-242, 2.65759088089109e-242], [4.8604817458834065e-76, 8.298060671892605e-08], [3.092976178087549e-33, 4.803484717282473e-08], [1.725058572419723e-33, 1.725058572419723e-33], [5.990062506468701e-08, 5.990062506468701e-08], [2.5857867752674893e-07, 2.5857867752674893e-07], [1.971548056464212e-12, 1.971548056464212e-12], [1.2219321549881348e-12, 1.2219321549881348e-12], [9.644627586551588e-117, 9.644627586551588e-117], [0.0, 0.0], [3.190951396404304e-84, 3.190951396404304e-84], [2.8995241716749122e-95, 2.8995241716749122e-95], [2.492391958779181e-95, 2.492391958779181e-95], [2.2868312088961338e-10, 2.2868312088961338e-10], [2.0344504665990389e-10, 2.0344504665990389e-10], [0.0, 0.0], [0.0, 0.0], [4.50587077942304e-12, 4.50587077942304e-12], [3.932200190630051e-12, 3.932200190630051e-12], [0.7016659635066052, 0.7016659635066052], [0.6916828737446885, 0.6916828737446885], [2.5621090133341397e-06, 2.5621090133341397e-06], [2.4842877480351613e-06, 2.4842877480351613e-06], [2.0356305736149548e-303, 2.8736500969512247e-72], [2.175024668298897e-239, 2.3539109900576066e-72], [0.0942080113601166, 2.4568603411857244e-21], [0.07019792592105928, 2.918564068581947e-09], [8.619927942227673e-09, 8.619927929958113e-09], [0.009015579070206791, 0.009015579070206791], [0.007799728280482938, 0.007799728280482938], [4.437688818930524, 4.43768391858037], [4.061351760627043, 4.061333080903454], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.4272697600418702, 0.4272697600418702], [0.6973314947592854, 0.6973306404877321], [0.0001666557118433584, 0.000130863300726506], [1.2355366447801096e-253, 1.2355366447801096e-253], [8.712188549313728e-84, 8.712188561714548e-84], [6.707794489745728e-84, 6.707794489745728e-84], [1.7694342086123616e-94, 1.7694342086123616e-94], [1.3563711183140714e-94, 1.3563711183140714e-94], [6.865760222520825e-10, 1.1348940444036376e-55], [5.93969568157703e-10, 5.341104185129549e-56], [0.0, 0.0], [1.3141189928099614e-11, 1.3141189928099614e-11], [1.1198977803004635e-11, 1.1198977803004635e-11], [2.295171645523452, 2.295171645523452], [2.2564207056846137, 2.2564207056846137], [7.986876747387988e-236, 8.592284829586608e-06], [5.284391614469241e-234, 8.003710107431427e-06], [1.4311869197739462e-71, 1.4311869197739462e-71], [1.147734576306162e-71, 1.147734576306162e-71], [0.18341398777282622, 1.2475159212977036e-20], [0.2313294638209773, 9.047144078394444e-09], [5.963933396682891e-05, 3.5664390882715946e-239], [7.115050238859024e-05, 2.5356733016400633e-239], [2.954601551747911, 2.954601551747911], [3.3202263713121702, 3.320226371312168], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [1.5407432752373669, 1.5407432752373669], [2.2344441445322594, 2.2344441445322594], [0.0, 5.0940071550926e-248], [1.7498892278750243e-84, 0.0], [5.831231831395027e-85, 0.0], [6.662630581475307e-95, 6.662630581475307e-95], [2.323739635459628e-95, 2.323739635459628e-95], [-2.147240960950102e-146, -2.14724096101889e-146], [5.440406039475859e-135, 5.440406039475859e-135], [6.847525295106078e-05, 6.847525295106078e-05], [6.980418516093355e-05, 6.980418516093355e-05], [6.876648865935325e-14, 6.876648865935325e-14], [0.0, 4.0698950385799954e-239], [2.1301063171445515e-241, -6.911158465394831e-241], [5.2679699253090187e-238, 5.2679333187708086e-238], [1.3494915613804594e-241, 1.3494521968757962e-241], [3.5458292920748707e-239, 3.5458292920748707e-239], [3.638470946487977e-72, 3.638470946487977e-72], [8.794475280665824e-73, 8.794475280665824e-73], [5.757002450131912e-240, -4.698035340474579e-258], [4.97738294951746e-239, 4.977382949520225e-239], [7.911726999830944e-240, 7.911726999830944e-240], [-5.20717587169268e-254, 5.4171125711556624e-244], [5.581306106644807e-240, 5.581623335942247e-240], [3.642128533419457e-255, 3.6421285323987084e-255], [2.099133026689194e-242, 2.099133026689194e-242], [2.2788147171533274e-242, 2.624926592557486e-244], [2.342871555457746e-242, 2.342871555457746e-242], [0.0, 3.194235240699364e-08], [0.0, 4.699253361423796e-09], [0.0, 2.0512415501286683e-34], [1.5370073452704152e-277, 7.578125908074054e-245], [9.683520001297138e-245, 1.3203299112582765e-17], [-3.946926547460478e-88, 4.7234545519079866e-158], [6.461746813358444e-26, 6.461746813358444e-26], [8.936647132295179e-104, 8.936647132295179e-104], [3.260189489629145e-104, 3.260189489629145e-104], [0.0, 0.0], [5.699438038739631e-85, 0.0], [6.114550223957932e-85, 0.0], [0.0, 0.0], [6.832742340793552e-96, 6.832742340793552e-96], [2.2926391274418436e-10, 2.2926391274418436e-10], [1.9884675672434302e-10, 1.9884675672434302e-10], [7.886073317618815e-05, 7.886073317618815e-05], [7.97161466128284e-05, 7.97161466128284e-05], [4.303100548566025e-12, 4.303100548566025e-12], [3.836663773951878e-12, 3.836663773951878e-12], [0.7011651003897392, 2.3055875857574353e-235], [0.6101810381763945, 5.697309907222084e-232], [2.9050124457503283e-06, 2.9050124457503283e-06], [2.4191004512981326e-06, 2.4191004512981326e-06], [0.0, 0.0], [5.4315325952057684e-73, 5.4315325952057684e-73], [6.9109776419925824e-09, 6.9109776419925824e-09], [4.3174660924443184e-09, 4.3174660924443184e-09], [3.1977201178942324e-89, 5.8454695546511326e-39], [0.0, 4.0807138685467704e-06], [0.0, 2.6042139486233466e-06], [0.0, 1.3493962558421775e-27], [0.00024169269968564875, 4.105184067837827e-48], [0.00016154424717194045, 3.6844065625804306e-21], [1.1289049987455222e-47, 1.5158762639392712e-20], [7.688837914530281e-13, 7.688837914530281e-13], [8.195004894478924e-13, 8.195004894478924e-13], [1.6102741913933067e-52, 1.6102741913932522e-52], [0.001997319565456395, 0.001997319565456395], [0.0013665778641644638, 0.0013665778641644114], [1.3410506968631124e-16, 1.1618086948628863e-38], [1.181000644187486e-06, 1.181000644187486e-06], [6.896428913976059e-07, 6.896428913976059e-07], [1.4458007261611373e-30, 1.4458007261611373e-30], [0.0, 0.0], [3.294534112520891e-84, 3.2945341078314923e-84], [2.227559511608428e-84, 2.227559511608428e-84], [1.668079438450617e-95, 1.668079438450617e-95], [2.53105552399183e-95, 2.53105552399183e-95], [1.716950509544935e-10, 1.716950509544935e-10], [1.6951516178074424e-10, 1.6951516178074424e-10], [0.0, 0.0], [0.0, 0.0], [3.275489442512656e-12, 3.275489442512656e-12], [3.2997436963486716e-12, 3.2997436963486716e-12], [-1.2968928726870578e-201, -1.2968928726870578e-201], [7.65276087232989e-07, 7.65276087232989e-07], [1.4956347680351712e-06, 1.4956347690996106e-06], [0.0, 1.7440415450446054e-72], [1.4504067812808845e-163, 4.1462381134529425e-72], [1.954820770033831e-31, 0.008734147126029265], [1.0340151562630014e-13, 0.008204194059056414], [2.016931774779193e-13, 2.016931774779193e-13], [3.2047879343786008, 3.2047879343786008], [2.428390899581373, 2.4209109257900483], [0.007053348927217898, 2.2432129789358303e-07], [3.563953358608137e-30, 3.563953358608137e-30], [3.553887630044843e-13, 3.553887630044843e-13], [4.425893543949453e-13, 4.425893543949453e-13], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.002291506613592362, 3.0913462218151967e-34], [0.0055033419906718, 9.465724408135527e-15], [4.665333376867335e-15, 4.665333376867335e-15], [6.281770369421902e-06, 6.281770369421902e-06], [3.895693459840728e-06, 3.895693459840728e-06], [7.545155007223734e-27, 7.545155007223734e-27], [0.0, 0.0], [0.0, 0.0], [3.90651990975743e-24, 3.90651990975743e-24], [2.5285338553661546e-161, 2.5285338553661546e-161], [4.461440986468455e-206, 0.0], [0.0, 0.0], [2.4095399567902553e-161, 2.4095399567902553e-161], [1.7737779721978232e-84, 1.7737779721978232e-84], [7.08201929305232e-177, 3.9379180421798475e-242], [8.690642818639708e-24, 8.690642818639708e-24], [0.0, 0.0], [0.0, 2.3141269062124256e-80], [1.3575026211808272e-162, 1.3575026211808272e-162], [4.558682984574733e-205, 4.558682984574733e-205], [9.735117203054426e-287, 9.735117203054426e-287], [1.0435139790959446e-162, 1.0435139790959446e-162], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [2.921031848233581, 2.9213853505755925], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [-1.8911267696182677e-61, -1.8911267696182677e-61], [1.0019761919664671e-48, 0.0], [0.0, 0.0], [5.764162950679859e-07, 1.903049379533676e-57], [4.964536577613881e-25, 4.964536577613881e-25], [2.0443420789209736e-26, 2.0443420789209736e-26], [6.353490565740629e-213, 1.0331279444844946e-22], [6.099315484103275e-94, 8.064530153407249e-23], [4.2093039622066567e-94, 4.2093039622066567e-94], [3.316921169082707e-11, 3.316921169082707e-11], [2.397043295922556e-11, 2.397043295922556e-11], [1.7889425924065225e-48, 1.7889425924065225e-48], [4.663909530820343e-05, 4.663909530820343e-05], [3.04575135543806e-05, 3.04575135543806e-05], [1.654829391470878e-24, 4.1401217130362445e-57], [1.5875276658685724e-25, 0.0], [4.417463946718532e-94, 0.0], [6.433773466405417e-97, 0.0], [1.1842290025239715e-110, 1.1177964826653792e-11], [1.5405548067580864e-48, 4.245463552915422e-12], [6.0160815440161294e-49, 6.016081544016561e-49], [8.578396153431258e-06, 8.578396153431258e-06], [3.4606475849469445e-06, 3.4606475849469445e-06], [3.0583377460177093e-25, 3.0583377460177093e-25], [0.0, 0.0], [1.0136809220932898e-22, 1.0136809220932898e-22], [1.0009806103893918e-22, 1.0009806103893918e-22], [0.0, 0.0], [0.0, 0.0], [44.908368874999994, 44.908368874999994], [1.1194113804007939e-286, 1.1194113804007939e-286], [0.0, 0.0], [7.782858878416956e-10, 9.447928360253192], [9.079683324122108e-10, 8.854118041204103], [61.155677834999985, 61.155677834999985], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [9.425357446733559e-14, 9.630099687393494e-14], [0.0, 0.0], [1.0170215992483946e-22, 1.0170215992483946e-22], [7.459366816738682e-23, 7.459366806121057e-23], [3.153271443417842e-213, 4.1397928801705215e-94], [3.139419101913906e-11, 2.028962821563687e-110], [2.1640327524419513e-11, 2.6354535555180908e-48], [1.6932276346918543e-48, 1.6932276346918543e-48], [4.533032196846323e-05, 4.533032196846323e-05], [2.601739720855462e-05, 2.601739720855462e-05], [1.5834156502488067e-24, 1.5834156502488067e-24], [8.522682260214675e-23, 8.522682260214675e-23], [8.708873295945435e-23, 8.708873295945435e-23], [0.0, 0.0], [5.382060152615526e-11, 0.0], [4.856465732921707e-11, 0.0], [0.0, 0.0], [0.0, 4.536481022274681e-05], [0.0, 4.331266051975525e-05], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [25.955647859462353, 4.013177275371116e-17], [1.0856460846761324e-36, 1.0856460846761324e-36], [3.5415942807150076e-08, 3.5415942807150076e-08], [9.251698041673879e-09, 9.251698041673879e-09], [1.5488433512351585e-113, 1.5488433512351585e-113], [2.0076303861222882e-27, 2.0076303861222882e-27], [2.4318088561499645e-27, 2.4318088561499645e-27], [2.6854070024825073e-105, 2.6854070024825073e-105], [1.4781293678075582e-28, 1.4781293678075582e-28], [1.5493803998255997e-28, 1.5493803998255997e-28], [9.935226435069954e-13, 5.792538673769505e-29], [0.03258581515742319, 0.03258581515578998], [0.009311034729060742, 0.00931103472906042], [1.1412270461480127e-56, 1.1412270461480152e-56], [3.2818903834093223e-13, 3.2818903834093223e-13], [2.3254549398349195e-13, 2.3254549398349195e-13], [5.580058965761976e-156, 5.580058965761976e-156], [4.967095668271503e-38, 4.967095668271503e-38], [5.182680392861043e-38, 5.182680392861043e-38], [3.102075268244499e-98, 3.102075268244499e-98], [6.0506325161238136e-83, 6.0506325161238136e-83], [2.569401165740038e-79, 1.2302386829654198e-84], [6.186489707821185e-10, 6.186489707821185e-10], [0.12045710310240472, 0.12045710310240472], [0.13878579439531094, 0.13878579439531094], [2.0130814955593036e-47, 2.013081492693908e-47], [6.963673871312728e-11, 3.307737356373068e-47], [4.2724378922599725e-11, 1.2390802956523342e-79], [1.3360063344829123e-183, 2.689579946556459e-189], [3.7929027605056084e-160, 3.7929027605056084e-160], [7.016980709006141e-99, 7.016980709006141e-99], [1.3160807320034414e-31, 1.3160807320034414e-31], [1.6143996116743363e-31, 1.6143996116743363e-31], [1.3349449031706074e-23, 2.0219409741653444e-224], [6.0900967454244696e-05, 1.1972196803236714e-164], [3.699062641436312e-05, 3.265462895294062e-153], [8.884686124368871e-88, 8.884686124368871e-88], [4.874738961194769e-88, 4.874738961194769e-88], [1.342626133134526e-236, 1.2541771299284105e-101], [3.713717956009499e-234, 8.521636111030937e-102], [1.7427176756308084e-06, 1.7427176756308084e-06], [1.6468030557343026e-06, 1.6468030545622773e-06], [0.0, 0.0], [0.0, 0.0], [7.417693680713916e-08, 7.417693680713916e-08], [6.134894458173074e-08, 6.134894458173074e-08], [6.291336509867398e-233, 6.37113847497149], [4.6644044787388e-234, 6.300982755005694], [0.0005873407472043828, 0.0005873407472043828], [0.0008004043751499923, 0.0008004043751499923], [6.648643086007632e-77, 8.869257165546731e-237], [3.903391832411624e-77, 2.2311875437565126e-234], [0.10283369683382654, 2.8300909422795875e-19], [0.07884812818531828, 1.6257372593854628e-08], [4.803998622635445e-19, 3.589444123267537e-08], [0.8775209719689225, 0.8775182282473424], [1.0248037592906494, 1.0246906673426321], [0.0002299730563255561, 0.00012973298298054296], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.05637709561976074, 0.05637709561976074], [0.050469255237699226, 0.050469255237699226], [0.0594226858522796, 2.1281676413483258e-20], [0.0698393229832987, 1.5460417537704626e-08], [1.7754098073077853e-08, 1.7754098073077853e-08], [0.0, 6.278636452573205e-249], [1.6761026323609434e-240, 7.831903178338305e-88], [8.8921886e-317, 2.6242738339910787e-88], [0.0, 0.0], [1.775435427719693e-240, 1.775435427719693e-240], [3.123765310815607e-07, 3.123765310815607e-07], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [1.3587624452831627e-08, 1.3587624452831627e-08], [0.0, 0.0], [0.35286859116764124, 0.35286859116764124], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [5.5687391746571226e-77, 5.5687391746571226e-77], [1.243799052962216e-77, 1.243799052962216e-77], [-5.734177389502377e-11, -5.734177389502377e-11], [1.7848292437460665e-53, 5.4019325565726605e-34], [2.0290517196625503e-31, 2.0283974291588653e-31], [-1.1350694036171785e-27, -1.1951264356357347e-44], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [-3.330184930587966e-17, -3.330184930587966e-17], [1.8418782046954592e-13, 1.8418782046954592e-13], [2.681653215260176e-13, 2.681653215260176e-13], [-8.668048278604536e-100, -1.3768457968317646e-29], [2.210760946216419e-23, 2.210760946216419e-23], [9.971113546592184e-23, 3.2480354053387395e-252], [0.0, 0.0], [6.08502568704077e-88, 7.236169319575445e-175], [9.81655039363058e-88, 5.233398832775243e-88], [1.2327267182014024e-101, 1.2327267182014024e-101], [9.496196968200056e-102, 9.496196968200056e-102], [1.7603271364752762e-06, 1.7603271364752762e-06], [1.7575522261259513e-06, 1.7575522261259513e-06], [0.0, 0.0], [0.0, 0.0], [6.956149081489472e-133, 6.790180233919385e-08], [4.99607533700505e-143, 6.795156702129773e-08], [11.312750825458394, 0.0], [11.29393459438755, 0.0], [0.0, 0.000808665397338796], [0.0, 0.0007911808253404884], [0.0, 5.980531008962801e-77], [0.0, 4.9415702367193456e-77], [3.042813464739274e-11, 1.1626091465283999], [0.0001130145869847298, 1.1046367382884639], [0.003050883790084574, 0.003050883790084574], [0.005505584858883081, 0.005505584858883081], [1.3910524477220006e-13, 1.3910524477220006e-13], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [5.134563603200142, 5.138892858604043], [4.894651281738555, 4.894651278255045], [1.55955593281754, 4.831974153254586e-30], [1.3716789711942918, 6.831880208943181e-19], [0.0007949576085105826, 0.00050293700912732], [7.690964383260524e-183, 7.690964383260524e-183], [1.8311127243976655e-87, 1.8311127243976655e-87], [1.2124072515161006e-87, 1.2124072515161006e-87], [0.0, 3.4393552304095665e-101], [0.0, 2.237415463973714e-101], [3.0870198418745345e-06, 3.0870198418745345e-06], [2.5883689608853677e-06, 2.5883689608853677e-06], [1.427640853128546e-237, 0.05847638403513524], [7.375557000414475e-239, 0.05056042547851285], [1.1810861866907542e-07, 1.1810861866907542e-07], [9.920809630363686e-08, 9.920809630363686e-08], [18.776011848134498, 2.3986671961844727e-238], [16.246703283580473, 1.7797267239664504e-239], [0.001391663770175422, 5.516611279763585e-240], [0.001165954386718012, 1.512191986998118e-239], [2.0709092186153894e-76, 2.0709092186153894e-76], [1.3914810569556073e-76, 1.3914810569556073e-76], [1.2936211116991057e-06, 1.2936211116991057e-06], [1.2063382222746862e-06, 1.2063382222746862e-06], [1.0056389286991385e-28, 1.0056389286991385e-28], [6.184625197154068e-06, 6.184625197154068e-06], [7.171006784585547e-06, 7.171006784585547e-06], [8.86323610770182e-58, 2.120178787248436e-25], [4.085198596589465e-06, 0.0], [4.3858825453934116e-06, 0.0], [2.774001680035804e-26, 0.0], [1.5130110264247916e-07, 7.053447424127178e-76], [1.1364857954731105e-07, 4.4888441558327734e-33], [3.1433670468585e-33, 3.143367043612405e-33], [4.562341686499005e-07, 1.5377800790011386e-69], [4.2461518995486746e-07, 1.6991183479616127e-30], [1.4411315482104414e-69, 1.7109026651585886e-30], [4.712627467488046e-12, 4.955722821618769e-116], [3.729378030365091e-12, 5.749405955277907e-51], [4.294640428789388e-51, 4.294640428789388e-51], [0.0, 0.0], [4.151678727383859e-88, 4.151678727383859e-88], [6.341338412894869e-102, 6.341338412894869e-102], [5.6975490882035375e-102, 5.6975490882035375e-102], [1.946114151903058e-06, 1.946114149132976e-06], [1.7394409730987034e-06, 1.7394409730987034e-06], [0.0, 0.0], [0.0, 0.0], [7.86802827185194e-08, 7.86802827185194e-08], [6.902009735482282e-08, 6.902009735482282e-08], [9.997405026155844, 9.997405026155844], [9.959546432110493, 9.959546439198673], [0.0007186902016487177, 0.0007186902016487177], [0.0006993640413460901, 0.0006993640413460901], [3.464596440054979e-77, 0.0], [2.877695056748189e-77, 2.5094956226266596e-239], [0.08710309651238388, 0.08710309651238388], [0.07942469111297984, 0.07942469111297984], [8.135129294543858e-09, 8.143780168071144e-09], [0.005670106745547057, 1.1400672508743941e-30], [0.0061945199229563475, 1.8408655969671912e-13], [4.4708473011190675, 1.4361468456175405e-05], [4.1543411485490696, 0.0429862571757197], [0.07520028513731031, 0.07576697866522428], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [9.85182618024191e-238, 4.7076165764125695e-33], [1.3625856762596772e-239, 6.875689607130024e-21], [1.2361203336276638e-253, 1.2361203336276638e-253], [1.6714121454994939e-87, 1.609835724638129e-235], [1.270081477455205e-87, 3.8471251641591866e-233], [8.306575496357116e-236, 3.8447744914008995e-101], [1.9931282322431217e-233, 2.907878465675639e-101], [5.164262160391276e-06, 5.164262160391276e-06], [4.759361013263131e-06, 4.759361013263131e-06], [0.0, 0.0], [0.0, 0.0], [1.9961858393781784e-07, 1.9961858393781784e-07], [1.8255162432784097e-07, 1.8255162432784097e-07], [3.50186597856418e-232, 1.4135380150950905e-235], [2.599500339480788e-233, 5.155594772056185e-238], [0.0022566488246077926, 6.282155444594757e-234], [0.0021513245216523815, 1.7202807711078288e-233], [1.9985266718814282e-76, 1.9985266718814282e-76], [1.558686115038897e-76, 1.558686115038897e-76], [2.1354287336427302e-20, 2.1354287336427302e-20], [1.548199408024987e-08, 1.548199408024987e-08], [6.651033992205382e-53, 3.483420359606712e-239], [2.2145761112582222e-23, 3.860944026314764e-239], [2.7074741676487326, 2.7074741676953473], [3.0779253885551676, 3.0779253885551676], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [3.021280395777745e-239, 1.3507262483344757], [5.244091473543217e-237, 2.0318126336175926], [1.2046112860872441e-237, 0.00011452234659612203], [0.0, 5.446261152633394e-248], [0.0, 7.355895445035102e-88], [0.0, 5.147069782660365e-88], [2.4243642456389063e-101, 2.4243642456389063e-101], [1.3897155265383896e-101, 1.3897155265383896e-101], [1.4794538630331895e-06, 1.4794538630331895e-06], [1.241145892923761e-06, 1.2411458911571206e-06], [0.019785661476385216, 0.019785661476385216], [0.01977222788385264, 0.01977222788385264], [4.964041663805936e-133, 6.409762462105338e-08], [2.370225751324871e-133, 5.1798032585702586e-08], [6.318328121509131, 6.318328121509131], [6.1491698725763735, 6.1491698725763735], [0.00047980400391048404, 0.0], [0.00044902049935131996, 0.0], [0.0, 1.0477585680446105e-76], [0.0, 6.478901988913649e-77], [2.5443268803028558e-17, 2.5443268803028558e-17], [2.18096121434153e-17, 2.18096121434153e-17], [6.976719320890002e-71, 6.976719320890002e-71], [1.6996394655783764e-09, 1.6996394655783764e-09], [2.90148411966488e-09, 2.90148411966488e-09], [6.022763002947501e-38, 6.022763001259552e-38], [4.406416533979615e-17, 4.406416533979615e-17], [4.105748794395661e-17, 4.105748794395661e-17], [1.4353879847327543e-69, 1.4353879847327543e-69], [7.648420063216177e-08, 7.648420063216177e-08], [4.810127108566111e-08, 4.810127108566111e-08], [4.912087836454407e-34, 4.912087839325499e-34], [5.943904458232238e-17, 2.581970925752799e-157], [4.682038169556598e-17, 1.5527215425251606e-69], [1.6735987854683356e-157, 1.2777045842846013e-69], [2.8420848594148533e-25, 2.8420848594148533e-25], [1.873241252849733e-25, 1.873241252849733e-25], [1.4432316443959205e-103, 1.4432316443959205e-103], [0.0, 0.0], [9.01529437498824e-89, 0.0], [7.913914880194905e-89, 0.0], [3.736408010262526e-102, 3.736408010262526e-102], [3.3048536344276714e-102, 3.3048536344276714e-102], [9.664028100142309e-133, 3.266426241760052e-06], [5.615706174169662e-133, 2.290015742679075e-06], [0.04105945338671586, 0.04105945338671586], [0.03160555023949355, 0.03160555023949355], [9.063513151702953e-133, 1.3349125856842107e-07], [5.1785004155525335e-133, 9.201984916117207e-08], [5.281366977139762e-232, 13.805289609279288], [3.928304647699741e-233, 11.081545858800006], [1.0314031066386503e-233, 0.0011764411745188018], [2.830050112049515e-233, 0.0008874540031149899], [0.0, 0.0], [0.0, 0.0], [6.7249093172224586e-09, 6.7249093172224586e-09], [4.5875085989043554e-09, 4.5875085989043554e-09], [5.7632805863367854e-39, 5.7632805863367854e-39], [0.0, 3.3216427264977736e-06], [0.0, 2.003797702366575e-06], [0.0, 1.1093272342766225e-63], [0.00021468198587966373, 0.00021468198587966373], [0.0001748748645389384, 0.0001748748645389384], [1.3454773860639236e-20, 1.3454773860639236e-20], [-1.1591764497505456e-66, 0.0], [5.633068409634707e-13, 5.633068409634707e-13], [1.193043200226131e-118, 0.0], [0.0016836826680626555, 0.0016836826680626555], [0.001437875929373141, 0.0014378759293730903], [1.1480714651525264e-16, 1.221646085096876e-38], [1.0053441932089205e-06, 2.8674550789249835e-71], [7.066697861997955e-07, 1.6080416268590526e-31], [1.54574877032502e-70, 1.2332122280329598e-30], [0.0, 0.0], [0.0, 4.4440258326540665e-88], [0.0, 1.4755385678299035e-88], [3.620608793687596e-102, 3.620608793687596e-102], [1.479338075597385e-102, 1.479338075597385e-102], [2.3633820324984044e-07, 1.6885889824248447e-198], [3.8369263134856906e-134, 2.4019608739243353e-198], [0.0, 0.0], [0.0, 0.0], [9.121150169375106e-09, 2.808277125675611e-199], [3.746767514814069e-134, 2.393170905531324e-198], [5.846390337580862, 2.0840908681884433e-200], [0.9314748644455463, 7.616435492964656e-203], [0.00015719604261976266, 0.00015719604261976266], [8.364420460790816e-199, 8.364420460790816e-199], [5.440361758146732e-79, 5.440361758146732e-79], [4.554452532594167e-78, 4.554452532594167e-78], [0.007714659730007933, 0.007714659730007933], [0.0016094209320100077, 0.0016094209306369263], [1.783452581052495e-13, 7.038175671679093e-32], [2.7971870775435658e-199, 5.432779924855561e-204], [4.759263944760282e-199, 4.759719840317726e-199], [-6.538894053776314e-203, 7.254834595074901e-204], [3.308284653695221e-30, 0.0010368779998935168], [3.2986520714311126e-13, 3.2986520714311126e-13], [6.556039878699816e-14, 6.556039878699816e-14], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0022617480144708213, 0.002261748014470821], [0.0004794110658651834, 0.0004794110658597485], [4.59822845282875e-15, 3.1881300561142517e-35], [9.135823953052748e-62, 3.932018153923216e-06], [6.647827384211594e-27, 2.3749158412676604e-06], [4.7260673906317975e-27, 4.726067542814417e-27], [0.0, 0.0], [2.9214647579151656, 2.9214647579151656], [3.2279183855313613, 3.2279183863485925], [0.27385579061872756, 0.273855790651549], [0.24935815802778818, 0.24935815802778818], [1.4434135649494523e-26, 0.40569135754279634], [1.0464931749993781e-14, 0.33777221413308267], [0.0, 0.0], [1.4404903264450432e-20, 0.34521261694529803], [1.706747656518933e-08, 0.32424918428503363], [3.5206722872586683e-08, 3.5233991680309107e-08], [2.7817672188759893e-05, 2.7826423724469368e-05], [0.0, 0.0], [4.120033349602763, 4.1200239631959565], [4.050991545767095, 4.050569617395902], [1.1796457977193938e-64, 1.1817290638289437e-64], [2.632639686181496e-43, 2.637288953823937e-43], [7.674859742753908e-15, 7.688413615249607e-15], [5.834088819050779, 5.834088819047955], [6.267238521373668, 6.266275309818041], [7.359828679986674e-11, 7.359816573024292e-11], [0.0, 0.0], [0.00036242342994495215, 1.52731517566689e-10], [4.688355805335263, 4.688355805335263], [4.588514237722543, 4.588006440306113], [0.3456455320613821, 0.34564553209700394], [0.2556389088397391, 0.25563892735469773], [6.396471265580739e-20, 6.396471265580739e-20], [4.63950126103654e-08, 4.63950126103654e-08], [3.7991789008606684e-08, 3.7991789040830134e-08], [0.31767338775166076, 1.5627908027739752e-20], [0.3536707180553197, 1.8517124765031684e-08], [2.930979261543117e-20, 3.776495385901986e-08], [4.3022070390526614e-05, 15.939270363934812], [0.12885464797545493, 16.061653965997294], [9.424262602696554e-11, 4.744672097772833], [0.000540022006566184, 4.1465767987797175], [0.0, 0.0], [0.7675502178652741, 0.7675502179643837], [0.8033690007633263, 0.8033690013350818], [5.855588651154183e-08, 5.868601299910687e-08], [9.865326413031967, 9.86527992623605], [10.590840021672697, 10.589228242281754], [7.4802355359120245, 7.4802355359120245], [8.154400908067766, 8.15440090726651], [0.0010674117951580797, 0.0010674117951580799], [6.105569965811729e-32, 2.191357381808997], [4.0924380615209264e-20, 2.143346515756414], [0.20490118547254532, 0.20490118547254532], [0.17608285409095975, 0.17608285409095975], [2.235907939895764e-08, 2.235907939895764e-08], [0.28584647638135063, 0.28584647638135063], [0.2465965555686572, 0.2465965555686572], [2.244482892203533e-05, 2.244482892203533e-05], [0.1881615330094605, 0.1881615330094605], [0.21192252327342087, 0.21192252327342087], [8.221873319452706, 8.221872796665814], [11.583206014042393, 11.52555236626584], [3.1655297576703854, 3.1655297576703854], [2.533521346014754, 2.533521346014754], [0.0002737598164845513, 0.0002737598164845513], [0.512990948692859, 0.512990948692859], [0.5365580102073221, 0.5365580102073221], [2.213165063576757e-05, 2.213165063576757e-05], [6.458452712114161e-10, 6.458452707517697e-10], [0.0, 0.0], [0.0, 5.119338106242521], [0.0, 5.555512622673151], [4.513403177636255, 4.513403177636255], [2.7780468055232657, 2.7780468055232657], [3.1588430130385716, 3.158843015286712], [2.8653272376430126, 2.8653272376430126], [3.30671408255099, 3.30671408255099], [4.757180216796982, 4.757180216796982], [2.775634398167477, 2.775634398167477], [3.1931166422516317, 3.1931166422516317], [3.1444405594126557, 3.1444405594126557], [3.718015738150652, 3.718015738150652], [0.3652490417698483, 0.3652490417698483], [0.22025739900609498, 5.746099650247303e-86], [0.23621336519413844, 9.257682612114881e-21], [0.2508887350352908, 0.2508887350352908], [0.28750344906199327, 0.28750344906199327], [0.3654945415922619, 0.3654945415922619], [0.215298767488134, 2.0986784963915762e-59], [0.23949441341454197, 1.0976021142316102e-08], [0.24378959392953892, 0.24378959392953892], [0.46811895224262084, 0.4681189368542601], [4.175752319012627e-08, 4.175752320490444e-08], [2.6018570677183715e-08, 9.078065379507974e-21], [2.6849454830105997e-08, 1.3421396759052487e-08], [0.4997303295286711, 0.4997303295286711], [0.3184951703250031, 0.31849517025898294], [0.34670248745057425, 3.7736247857619204e-20], [0.36219844015606706, 0.36219844023185316], [4.8684209218448475e-20, 0.6041529567368523], [0.46702661390718175, 0.4670265664363201], [0.26277828524219304, 0.2627782540239461], [0.29424054666519667, 2.7534200812234304e-08], [0.3304998725888208, 0.3304998725888208], [3.547490586429197e-08, 0.550250647079951], [0.07323595846925417, 0.07323592029170092], [2.4467943813541863e-08, 0.04412128636963044], [0.04758111864841355, 2.6533410569591714e-08], [0.05522598063936722, 0.0552259806818195], [0.05629083367158968, 0.056290833671621195], [0.3690467425091619, 1.7571263024776396e-20], [0.23705722432683046, 0.23705722432683046], [0.23598522580966363, 0.2359852259776137], [0.2722880781638831, 1.2395829647608506e-20], [0.2770501968562552, 1.2989336573329723e-20], [0.3778430553074599, 2.0819748430631818e-08], [0.18871342192688845, 0.18871341055064278], [0.19402574354533383, 0.1940257325594312], [0.2783932829984066, 1.4685706692611384e-08], [0.292967639887971, 1.5380963633135772e-08], [1.4260891204133696e-08, 2.974719921304029e-08], [3.212631201419936e-08, 1.5743361076436895e-08], [18.010676990725983, 18.01067699108854], [8.376579956434048, 8.376579956434048], [10.263997136269927, 10.263997136269927], [12.34748614728488, 12.34748614728488], [20.684359651107105, 20.684359651107105], [19.16936836357574, 19.16936836717074], [8.780020661182782, 8.780020661182782], [10.713036530486905, 10.713036530486905], [13.044469411378332, 13.044469411378332], [21.411471955015426, 21.411471955015426], [5.14130894746189, 5.141295064628329], [2.6304265816082286, 2.6304265816082286], [3.011590080667378, 3.0115974650487005], [3.954191219799658, 6.800699800583301e-11], [9.983203103618276e-11, 5.586743387379985], [4.679418693080007, 4.679418693108402], [2.2952059724889233, 2.295205972506758], [2.7010334418855946, 2.701365420856479], [3.4737267652460826, 0.0003897832769363214], [0.0005725173724483044, 5.148861018732838], [0.0010605476835485257, 0.0010516164841351304], [0.0006122227781083291, 0.0006122227781083292], [0.0004459704040152213, 0.0007110299190276235], [0.0011272995357710196, 0.001142205008830583], [0.7581545403489595, 0.7581545397079099], [0.5032389546863314, 0.5032389546863478], [4.2869075635787016e-20, 0.5580817198928106], [0.5807793907866806, 0.5807793907866806], [5.0706373014024474e-20, 0.7694357453080828], [0.7468445340660261, 0.7468445340542089], [0.5017013676608914, 0.5017013676764356], [3.10755358020375e-08, 0.5319348675307388], [0.5674650226960959, 0.5674650226872033], [3.6758641620039996e-08, 0.7794600197014129], [0.07275826946438692, 0.07275826945637806], [1.2181474542817505e-11, 0.04783777986211587], [0.04214326506716582, 1.3199485421111882e-11], [0.054707896490791684, 0.054707896490791684], [0.054286563835745226, 0.05428650515437962], [8.937225584664759, 8.937225584664759], [5.468871197238599, 1.636129204177051e-228], [5.885242854021599, 5.885242854021599], [6.454811062126848, 6.454811062126848], [9.245065966742635, 9.245065966742635], [9.340048769072757, 9.340048769072757], [5.95090304516864, 4.7013649738639606e-229], [6.375836988317206, 6.375836988317206], [6.863643387120716, 6.863643387120716], [9.815482158186867, 9.815482158186867], [7.083409645506984, 1.0885714703467332e-10], [4.478897620007535, 4.478897620017429], [5.176688367569858, 5.176688367569858], [5.201553060341152, 5.2015530603408635], [7.501605368732177, 7.501618113845121], [7.574320414210931, 0.0006244408834490561], [4.750009410084779, 4.750009410084779], [5.069775426376164, 5.069775426376164], [5.6840148840086355, 5.6840148840086355], [7.774458026773707, 7.774458025810893], [0.0007886861991632554, 0.0007886861991632554], [4.987699727998244, 4.987699727998244], [2.693153379176927, 1.844011902399904e-229], [3.1332257825017815, 3.1332257825017815], [3.7891170292528833, 4.2155823858157895], [1.8674624404702882, 1.8674624404702882], [4.558973126647138, 4.558973126647138], [2.471483345441094, 7.796492298449158e-230], [1.450111461927072, 1.450111461927072], [22.919751211722378, 22.919427095676745], [1.8557603108884386, 1.8557603108884386], [0.379230210641956, 0.379230210641956], [0.21205126693339815, 0.2120512671035534], [0.18641244028637766, 0.18641244028079443], [0.2877321102070358, 0.2877321102070358], [0.4038921466706635, 1.736887108841396e-20], [0.3475581268061625, 0.3475581482529331], [0.1355282642339062, 0.1355282642339062], [0.13611623495072647, 0.13611623495072647], [0.25102957697574374, 0.25102957697574374], [0.1582374954093365, 2.05946350159266e-08], [3.583447288349775e-20, 4.3342775574300777e-08], [2.3028916407679956e-08, 2.30233208107029e-08], [44.19278621765322, 2.1353672578185757e-08], [0.5257849291634946, 0.5257849291634946], [0.3212797788607276, 0.32127977886075854], [0.3419117917028991, 0.3419117917028991], [0.4429233238052129, 0.44292332380519134], [0.5306618758769334, 0.5306618759724973], [0.451639144126291, 0.4516391922338258], [0.17734850110130132, 0.17734850110130132], [0.2865240801763125, 0.2865240801687661], [0.35282199439508644, 0.35282199439508644], [0.18651095167809442, 0.18651095167809442], [2.942632793434186e-11, 0.08235061911558528], [0.044789674389872226, 2.460442382988603e-08], [2.6149999369500004e-08, 0.04646191907445222], [0.06245762032140926, 0.06245762032140926], [4.077359672021786e-08, 0.035941905866435225], [0.38207150598699396, 0.38207150548443247], [0.19262383610298414, 0.19262383608373695], [1.1482744693657686e-20, 0.18928635560143997], [0.2730690063592765, 0.2730690063592765], [0.3731896170390956, 0.3731896170390956], [0.3974962589865096, 0.3974962804412923], [0.18648513969055047, 0.18648513969055044], [1.3595435300628659e-08, 0.19902376654856269], [0.2975448153138152, 0.2975448153138152], [0.36957046538873134, 0.36957046538873134], [48.80387257730276, 2.1379047031399208e-08], [3.1938564618114874e-08, 3.193856461811488e-08], [2.1204523646594803e-08, 4.42177788353468e-08], [19.36474740357498, 19.36474740357498], [9.564992061470733, 2.876295877624209e-232], [9.627468652372672, 9.627468652372672], [13.300729769783736, 7.400340631264695e-11], [18.502235339059332, 18.502235339059332], [19.435580000503567, 19.435580000503567], [9.524236618361625, 3.1635408139214686e-232], [10.162920406711203, 10.162920406711203], [13.757408477680526, 1.8850726468457847e-07], [19.01297532435858, 19.01297532435858], [5.2266904418028, 5.2266904418028], [2.5858866766554205, 2.5858866766554205], [2.6938177493745963, 2.6938177493739714], [3.535070037997293e-47, 4.132631364551966], [4.812502240480433, 2.4996647137906077e-32], [4.745435815093139, 4.746049282260171], [2.286023102790647, 2.286023102790647], [2.4117748524756752, 2.4117748513649957], [2.612160233518646e-32, 3.5453388523189924], [4.3868421839249585, 3.6650958855164034e-20], [0.000661928239133969, 0.00112722354375085], [0.0004163601195905041, 1.194692080629151e-10], [0.0006583248339213447, 82.00122261032843], [1.1049218281625684e-10, 0.0005840636677552837], [0.0006230134369064116, 0.0006082149844620526], [0.774184245107483, 0.7741842452215852], [0.5063341329675844, 0.5063341329675844], [0.5211267439929683, 0.5211267439929683], [0.6697362273358289, 0.6697362273358289], [0.7195132714469624, 0.7195132714469624], [0.7530218010692157, 0.7530218010692157], [0.5371095977463463, 0.5371095977463463], [0.5502678342256159, 0.5502678645528353], [0.698451758366991, 0.698451758366991], [0.7500681682596287, 0.75006816824824], [5.9127592959077864e-08, 0.07368011083838875], [0.04412244461734454, 3.867024922375654e-08], [1.3015862142391321e-11, 0.045914184995554136], [0.061464583010675033, 0.06146458301066082], [5.4900646935400086e-08, 0.06967010673199357], [0.0, 9.153394757721129], [6.660323371162907, 6.660323371162907], [6.893155163357547, 6.893155163357547], [8.607411891090358, 8.60737284735213], [9.476064867110635, 9.476064867110635], [0.0, 9.581859119148158], [7.268428989176837, 7.268428989176837], [7.459975580949384, 7.459975580949384], [9.147361529184616, 9.147361529175864], [10.096498064108932, 10.096498064108932], [7.699621675127088, 7.699635324285636], [4.852963208160637, 4.852963208160637], [4.967859335897954, 7.377613696424295e-11], [6.794722739570604, 6.794722734734822], [7.084350280119528, 7.084339003051572], [7.7785048169804325, 7.7785048169804325], [5.505941506444373, 5.505638719497947], [5.620202592739938, 0.00042304716233742987], [7.200076144141799, 7.20054708792216], [7.718183112573159, 7.718183112573159], [0.0006584157690801645, 80.11710292539738], [3.0023072572123892, 3.0023073044173882], [3.465352141993759, 3.465352141993759], [1.2237029896629845e-19, 1.913858725724532], [2.0606867240430207, 4.209239855934013e-230], [0.0, 2.534378054832607], [3.432045810882592, 3.432045810882592], [5.74662721532559e-14, 1.723181030069457], [2.0742436888621936, 1.2793142952973172e-229], [0.0, 2.173320472495479], [0.25897956575437464, 0.25897956575437464], [0.28862880052409606, 0.28862880052409606], [5.052741348105478e-21, 5.052741348707731e-21], [0.12626273830076437, 0.12626273830076437], [0.32671922989438285, 0.32671922991065916], [0.24020633592966623, 0.24020633592966623], [0.27730325155396307, 0.27730325155396307], [5.9901037057292475e-09, 5.990103702180087e-09], [0.1538481838778858, 0.1538481838778858], [0.17026653418001902, 0.17026653418001902], [3.165652438071738e-08, 3.165652438071738e-08], [0.3868109614251695, 0.3868109615022405], [0.45293524896854603, 0.45293524896854603], [0.20667183052734905, 0.20667183052736043], [0.25111359492999497, 0.25111359492999497], [0.26117534523823915, 0.26117534523823915], [0.31602320234704506, 0.31602320234704506], [0.381383045653421, 0.381383045653421], [0.16836646827638377, 0.16836646827638377], [0.1900579596977962, 0.1900579369954876], [0.23219038448054904, 0.23219038448054904], [2.9563960065036017e-08, 0.056665097214451995], [0.0616112413138563, 0.0616112413138284], [0.03289939273496364, 0.032899392734963645], [0.03586542306499248, 0.03586540394627583], [0.03691143895682467, 0.03691143895684264], [0.31768977263522385, 0.3176897726060926], [0.26789032634832116, 1.2825074487590693e-20], [0.11452260844929758, 0.11452260844929758], [0.12416791039395268, 0.1241679103825356], [0.3041105182155614, 0.3041105182318076], [0.31321523019058317, 0.31321522974475424], [0.2922624097705777, 1.519550963461668e-08], [0.11598662850554597, 0.11598662850554597], [0.123145524885808, 0.123145524885808], [0.10669537263834938, 0.10669538108990607], [3.2852572960298926e-08, 3.281829949323228e-08], [1.2632006990982887e-08, 1.2632006990982887e-08], [1.4256468064931353e-08, 1.424299738820634e-08], [1.8567841478333568e-08, 3.584623310650053e-08], [12.780606677148873, 2.879752859771314e-05], [13.027227858039783, 13.027227858039783], [4.567884412524412, 4.567884412524412], [5.733575910451956, 5.733575910451956], [14.475747971285578, 14.475747971285578], [13.5879497393494, 0.08619281695529336], [13.353710231214496, 13.353710231214496], [5.358058271426269, 5.358058271426269], [6.20996534102179, 6.20996534102179], [15.210289157655401, 15.210289157655401], [6.816811881296848e-11, 2.750137886588483e-47], [4.099925635326091, 8.146892932414495e-11], [4.013291635135948e-11, 1.8495810057520796], [2.2318477601817113e-47, 1.9312077674433479], [4.111265571599472, 4.111265571599472], [0.00039088041719260736, 2.0315836384772333e-32], [3.523932441455772, 0.0004669225103592108], [0.00023033909072423737, 1.5396843915712046], [1.650329098715156e-32, 1.6213066427343226], [3.6787923662345143, 3.6792488108785135], [0.0003457859003244554, 8.597636114162791e-11], [0.000360819896423944, 0.0003540982077772592], [0.00015966737466273446, 0.0001625986352040059], [6.98259202028783e-11, 0.00016671565839116313], [1.6514439837496772e-10, 0.00035461748477430706], [0.6008358136952348, 2.4941506496606466e-23], [0.6625342982234312, 0.6625342982234101], [0.3789623051713932, 0.37896230512582996], [0.3914883685741747, 0.39148836852448965], [3.321548144196879e-20, 0.6339697660737331], [0.5792454107203296, 1.8080111749424992e-11], [0.689767697033957, 0.689767697033957], [0.4108000219359491, 0.4108000219359491], [0.42992105296878347, 0.42992105296878347], [2.410238684940867e-08, 0.4457944701591961], [4.590214063564989e-08, 0.05564656945317903], [0.06083853749531266, 0.06083853749531266], [0.03395236744212371, 0.033952367375552975], [0.03481859784547809, 0.034818597775783014], [0.03670860974976734, 0.03670860974976734], [6.5795020579636745, 6.579502058185541], [8.52551213220319, 8.525512144338359], [5.011386202213807, 5.011386202213807], [5.294829419785848, 5.294829419785848], [0.0, 1.8549437545148916e-251], [7.007207512443574, 7.007207512538501], [9.054653851795933, 9.054653851795933], [5.520697420097429, 5.520697420097429], [5.790571640368248, 5.790571640368248], [0.0, 1.9452766629849971e-236], [6.817135972357501e-11, 6.817135972357501e-11], [1.4876276066505555e-32, 8.144733196414636e-11], [4.021238671734546e-11, 6.708018883987193e-33], [4.2867013638258006e-11, 6.878420088853651e-33], [7.965628951356645e-11, 7.965641054805091e-11], [49.06510952452668, 0.08335038486443855], [3.951798931659137e-16, 6.709825023603514e-10], [3.120005580155574e-10, 3.120005580155574e-10], [3.456571759117839e-10, 3.4565717591178387e-10], [6.970675240291124e-10, 6.970675240291124e-10], [7.31295247144323e-17, 7.31295247144323e-17], [6.838693118664254e-17, 6.838693118664254e-17], [5.4232493927785044e-71, 5.4232493927785044e-71], [2.7447204493752798e-06, 2.7447204493752798e-06], [3.038674435452216e-06, 3.038674435452216e-06], [2.13018116713989e-28, 2.13018116713989e-28], [0.2866833857701165, 0.28668338568642493], [0.26357414981686456, 0.26357414981686456], [3.112311232069075e-08, 3.105501879142479e-08], [5.893239774062457e-17, 1.0185754493176884e-159], [7.413571454560179e-17, 1.896055965383324e-70], [1.6159501971129938e-70, 1.6159501971129938e-70], [2.752348753490777e-25, 2.752348749573099e-25], [2.927792215101728e-25, 2.927792215101728e-25], [5.2345826340089193e-104, 5.234582645185254e-104], [3.7148119171132756e-17, 3.7148119171132756e-17], [3.0632860841220945e-17, 3.06328607976183e-17], [2.75363190249994e-71, 1.3795913956402185e-161], [3.2590876428578715e-65, 1.978229686095102e-06], [2.302141682195625e-28, 1.7285383618125007e-06], [1.5371344059597233e-28, 1.5371344059597233e-28], [0.2313573466612638, 9.068515839009383e-20], [0.1866259414608744, 3.6645866869512996e-08], [2.5089115650904044e-08, 2.5089115650904044e-08], [2.4298125755990273e-17, 2.4298125755990276e-17], [2.014622173770581e-17, 4.01823629297468e-17], [6.671446565518949e-71, 5.911484573757108e-160], [2.4325501567675377e-25, 2.4325501567675377e-25], [2.2158692229233476e-25, 2.2158692229233476e-25], [4.6268517529913515e-104, 4.6268517529913515e-104], [5.248511226409558e-26, 0.0009865674977402995], [0.0010152631699321488, 0.0010152631699322143], [0.001385018770171719, 0.001385018770171719], [0.0009770969305027217, 0.0009770969305027217], [0.013569361453535128, 0.013569361453535128], [0.0014728206580277192, 0.0014728206580277192], [0.2755404603585461, 0.2747094635620019], [0.2544904431436466, 0.2544904823333795], [3.370880622302935e-08, 0.0012169671155889523], [5.796715423840504e-17, 0.0007032728731401955], [0.0006486941054439579, 0.0006486941054439579], [0.0006480223474701019, 0.0006480223474701019], [0.0007946687281725029, 0.0007946687281725029], [0.0007501526423149194, 0.0007501526423149194], [5.150008451731341e-71, 0.0008578418661089385], [5.908526427592354e-17, 0.0008553197548697187], [0.0011175431539442494, 0.001117543153944192], [0.0014774249276466411, 0.0014774249276466411], [0.014126431609271047, 0.014126431609271047], [0.0010247312580875007, 0.0010247312580875007], [0.0015138024079876652, 0.0015138024079876652], [0.2934542169159769, 0.2926359294875966], [0.25597504303163476, 0.2559711056905353], [3.162732948655234e-08, 0.0011747584785158558], [1.5170796248161269e-06, 0.0007364454402141462], [0.0006440092232943962, 0.0006440092232943962], [0.000719889024821997, 0.0007198890243096538], [0.0007338290948549832, 0.0007338290948549832], [0.0007420155675455295, 0.0007420155675455295], [5.2740504124777816e-104, 0.0008410079765919855], [9.327756154860519e-11, 4.836621257981743e-103], [5.28002772613886e-11, 3.3453786231834153e-45], [1.8415101645981174e-45, 1.8415101645981174e-45], [5.948407389358487e-06, 5.948407389358487e-06], [3.4079520018436966e-06, 3.4079520018436966e-06], [4.026845450963871e-26, 5.626559413824535e-60], [0.27361589522061597, 0.27361589522061597], [0.194316774141581, 0.19431258938710205], [2.3589186585958903e-06, 6.194950894567167e-15], [1.9535840478251783e-17, 1.9535840478251783e-17], [1.5884522827757905e-17, 1.5884522827757905e-17], [6.35427799341376e-70, 6.35427799341376e-70], [3.7880385031581395e-27, 3.7880385031581395e-27], [2.2660543938396276e-27, 2.2660543938396276e-27], [6.664562386798606e-110, 4.752696594018431e-248], [7.009099333905524e-11, 7.009099333905524e-11], [4.483727941337741e-11, 4.483727941337741e-11], [1.3804234176855125e-45, 1.3804234176855125e-45], [5.754076163701558e-06, 5.754076163701558e-06], [3.508974908502311e-06, 3.508974908502311e-06], [5.797598907408397e-60, 3.906612411897364e-26], [0.729195942807783, 0.729195942807783], [0.45558260932260963, 0.45557151686057346], [6.293131212950983e-06, 1.451751000171049e-14], [1.0719108245981342e-156, 5.213126422845432e-17], [3.2251828040051565e-69, 3.869322557723223e-17], [1.6976188984869042e-69, 1.6976188984869042e-69], [9.553184651612097e-27, 3.0785923113582234e-247], [6.459536076979925e-27, 2.5835708217469803e-109], [1.6808890736768133e-109, 1.6808890736768133e-109], [0.00037781205630690873, 9.733008236258614e-11], [0.0005896610818115871, 0.0005896614451336419], [0.0010237336872408632, 0.0010237336872408632], [0.0003813388890473474, 0.0003813388890473474], [0.4440696249146499, 0.0005947321826456442], [0.0010390461527734127, 0.0010390461239387382], [0.442109548975241, 0.442109548975241], [0.3491392980500623, 0.31429467838723785], [0.002283073729691112, 0.003032506328743411], [0.00021161681797621067, 0.00021161667253667685], [0.000277885426713936, 0.00027788530601414926], [0.00045261915264429, 0.00045261915264429], [0.0003335012108878919, 0.0003335012108878919], [0.000398704113214756, 0.000398704113214756], [0.0006032852575853687, 0.0006032852575853687], [0.0003377319935851463, 6.40696476925145e-10], [0.0005594639695234469, 0.0005594639695225332], [0.0009925814085316018, 0.0009925814085316018], [0.6257116508399434, 0.0003327997779088206], [0.0005366783405982085, 0.0005366782930973228], [0.0009732404810567462, 0.0009732404810567462], [0.6382527401415162, 0.6019747622577853], [0.43020872145163824, 0.4302087214516383], [5.18744411145463e-06, 0.0005663405214626208], [0.0001608220416758733, 0.00016082204167587334], [0.00017121388110235097, 0.0001712138811014127], [0.00033253097170155054, 0.00033253097170155054], [0.0002296792296289372, 0.0002296792296289372], [0.0002580250563609152, 0.0002580250563609152], [0.0004407141151714594, 0.0004407141151714594], [5.270553173042829e-15, 5.270553173042829e-15], [4.901440240865706e-15, 4.901440240865706e-15], [1.1623749548372493e-63, 1.1623749548372493e-63], [8.564299976621042e-12, 8.564299976621042e-12], [7.89232543078175e-12, 7.89232543078175e-12], [7.819741643141399e-51, 7.819741643141399e-51], [0.004602014089877571, 0.004602014089877571], [0.00366353362358862, 0.00366353362358862], [5.408995001886536e-16, 5.409100490097096e-16], [6.706792180192889e-234, 4.846108046032507e-25], [2.0723208713383464e-103, 5.41568364632102e-25], [2.466872037785095e-103, 2.466872037785095e-103], [6.966296787023758e-39, 0.0], [7.876335789303197e-39, 2.4825339968613815e-296], [1.1999939662503945e-290, 5.769894045631026e-305], [1.177827423310198e-144, 5.306361955195823e-15], [1.4322059324888877e-63, 5.260143068835873e-15], [1.185347863022369e-63, 9.663238542966418e-145], [7.55350040828681e-116, 8.94130042280073e-12], [8.758115071221429e-51, 8.657507787290331e-12], [8.182446158517641e-51, 6.833024375279246e-116], [1.714999757899154e-37, 0.004851384017405804], [6.071023084489907e-16, 0.0046127008147846165], [5.707605003775064e-16, 1.533048304299986e-37], [5.848627202551856e-25, 7.454219632115649e-234], [6.403536290973656e-25, 2.3000826775734507e-103], [2.9691706407919463e-103, 2.9691706407919463e-103], [1.0420388882047283e-38, 1.0420388882047283e-38], [1.0384691954798792e-38, 1.0384691954798792e-38], [1.4759432241359099e-158, 1.4759432241359099e-158], [0.0014102589684212444, 5.524481503896695e-15], [0.0014928888141429889, 4.294664103060117e-10], [0.0016955502098336824, 0.0016955502098336824], [0.0014418400630253534, 9.079220254327807e-12], [0.012193652035965268, 0.01375172472620347], [0.001712174021920691, 8.299606514714745e-51], [0.004010212333626072, 0.0013972644013820982], [5.143008596013982, 5.140351411913702], [0.001524050870343216, 0.0014773432584303282], [0.0012423752055680999, 7.321369530966568e-234], [0.0011462048992980196, 2.2600995105275836e-103], [0.0009514523162583534, 0.0009514523162583534], [0.0013336581163839506, 0.0013336581163839506], [3.409874249352077e-19, 0.0012750660049730379], [1.0511517163433488e-114, 0.0010682310283775798], [0.0013239985388878244, 4.311578294921171e-10], [0.0014149778587695244, 5.500848128736124e-15], [0.0016426309427698305, 0.0016426309427698305], [0.013645548908177206, 0.012303101136776927], [9.006546373442266e-12, 0.0014555590723857763], [0.001839985500121555, 1.7041299326572324e-83], [4.688378048780897, 4.685049448758807], [0.004443186568049868, 0.0012734738059271677], [0.001283955431354862, 0.001283955431354862], [0.0011394740803840151, 1.2812796011748457e-176], [0.0010360645872474579, 8.192267566316052e-67], [0.0008660281590313529, 0.0008660281590313529], [0.0012020745658075452, 0.0012020745658075452], [9.64457348426471e-39, 0.0011113980331872468], [1.3060783564527185e-158, 0.0008972071629958101], [3.221178318669902e-28, 0.0], [5.315177956008931e-28, -1.358763587711783e-229], [2.3359394325277296e-114, 4.271791364660179e-114], [2.8929918407086148e-28, 2.8929918407086148e-28], [2.9785670286764725e-28, 2.9785670286764725e-28], [0.0, 0.0], [0.0, 0.0], [2.633453815480408, 2.633453815480408], [2.2195044511241075, 2.2195044520248772], [0.0006198493083831902, 0.0006198493083831902], [0.0, 0.0], [27.38803791535141, 2.788346437051039e-05], [135.77917900603114, 0.08347293227573956], [0.19322328597125416, 0.19322328597125418], [0.0, 0.0], [3.9645769994603532, 3.9645769994549633], [3.3102209910540017, 3.3102209850466506], [0.0018286595871843123, 0.0007431157485210268], [0.0, 0.0009510546354883223], [2.566058007247245, 2.5660235726217504], [2.744057666823838, 2.744363402809765], [1.0798512601638174e-10, 0.001273544728596], [0.0010713912032275044, 0.0010713912032275044], [9.679510762092425, 9.679476062939898], [10.437453265518796, 10.43817293212223], [0.0724697835200803, 0.07166216399042528], [0.0, 0.002304271298384917], [0.0008491864587613926, 3.9020431820596437], [0.6925609734272955, 3.8746167611146385], [2.3245766048495193, 2.3241789120022345], [0.0, 0.0008299896430115189], [2.8453453425566604, 2.844506068708958], [2.8456548234268992, 2.846460836528991], [0.0003173292061545829, 0.0015055448212135916], [0.0008855141398587804, 0.0008855141398587804], [10.519162680930862, 10.527829797339797], [9.876200241861536, 9.963024457941128], [0.09546223138714222, 0.1811576346955085], [0.0, 0.0020106178932290096], [4.748456284638902, 0.8705552145516787], [6.382361531351224, 3.146106736761179], [0.003359632529780038, 0.0019694958015060594], [0.0, 0.0], [1.3663680954547864e-10, 4.504776476081183], [0.0005077759149841011, 4.47931328194197], [0.00102594550316996, 0.0005147377576969335], [0.0, 0.0], [17.28450637832565, 4.3073344040481724e-05], [18.794100607491067, 0.12892971691852884], [0.24632867612829876, 0.25825079559892206], [0.0, 0.0], [6.85246807560004, 6.852514560096832], [6.001894147766041, 6.001894147449885], [0.00317710618321724, 0.0013154192820726946], [0.0001148852167176267, 0.0001148852167176267], [0.0, 0.00012042465862485684], [3.8951882202623773, 4.453932972266737], [26.803048629531336, 26.827148559266234], [26.22388689654527, 26.861193298255444], [0.00011342070052566162, 0.00011342070052566162], [11.016116204670423, 11.016116212507109], [11.264148748245486, 11.182413361594685], [0.08214469230736422, 0.00013170805929568535], [0.000278007938248066, 0.000278007938248066], [4.684665631783794, 0.0002132758306149291], [3.922288166277885, 0.0013564455732350243], [0.0011035872309120742, 0.0008797470084155157], [9.308623499871039e-05, 9.308623499871039e-05], [0.0, 9.204548142295366e-05], [3.4445724159751085, 3.4236391903676187], [4.280758001698626, 4.296745820428106], [19.39812889916834, 19.913742200780018], [9.40980825911154e-05, 9.40980825911154e-05], [13.643159645949009, 13.653268128499057], [13.855007486954174, 13.95559503168096], [0.09897722283857956, 0.11206919085626325], [0.0002472693379112525, 0.0002472693379112525], [5.517982421087052, 5.517881198320732], [4.71710842905911, 4.717234931394145], [0.002865912028982377, 0.0026374770800830167], [44.908368874999994, 44.908368874999994], [0.0, 0.0], [2.151811311777362, 2.151811311777362], [2.1259273543751718, 2.125927354359101], [0.0002401061872133606, 0.0004846888076656899], [0.0, 0.0], [8.04986289191523, 1.9270668231801277e-05], [8.417375245901999, 0.05768388609657801], [0.11834906054181596, 0.05982415141219452], [0.0, 0.0], [3.8751258253076277, 3.8750966639473647], [3.1097483162724973, 3.1097483162724973], [0.0007438912340980745, 0.0007272843082564736], [1.3227953682732703, 1.3228117104629995], [0.9339506349486781, 0.9344711765466868], [2.257369536995855e-10, 0.00032752727485738065], [1.0137673109225325, 1.0137673109225325], [0.6663090369179034, 0.666364564519317], [1.8308357741355865e-12, 4.6303849004022854e-05], [9.882734200378792e-26, 9.882734200378792e-26], [7.453137405038658e-26, 7.453137405038658e-26], [8.74724055460496e-105, 8.74724055460496e-105], [1.1245442803743263e-23, 1.1245442803743263e-23], [8.849104275823624e-24, 8.849104282121476e-24], [2.638058877351311e-96, 1.8754897510301403e-217], [4.521797239797639e-27, 5.717117701467992e-133], [3.3560994974817835e-27, 4.3999608452021983e-162], [3.192185675866354e-110, 1.0032043120299737e-132], [8.404148907725025e-18, 8.404148907725025e-18], [6.67679703237883e-18, 6.6767970228751124e-18], [0.022562513740437058, 0.022562513740437058], [0.013497920453788956, 0.013484442521878214], [1.2272986152283135e-12, 1.2272986152251607e-12], [4.9374278002716247e-20, 9.128943136293284e-183], [4.3686238266166726e-20, 1.0074549017902676e-80], [2.9245524307131565e-81, 2.9245524307131565e-81], [8.424836399791322e-20, 8.424836399791322e-20], [2.8593160982401675e-19, 2.8593160982401675e-19], [2.6716294330546128e-20, 2.6716294330546128e-20], [2.796412638806371e-20, 2.796412638806371e-20], [5.742252972466446e-82, 1.0116507965706878e-184], [1.9705272235745416e-15, 8.669382274636998e-07], [1.653224481675896e-15, 1.6532244828524856e-15], [1.87312748386001e-08, 1.87312748386001e-08], [1.2112935521720469e-08, 1.2112935521720469e-08], [4.5997013683172554e-36, 4.5997013692498745e-36], [0.01559635855270809, 0.01559635855270804], [0.01038148288329538, 0.010381482890313412], [3.5869878688812523e-13, 4.5612134006760834e-31], [5.79889016351951e-07, 5.79889016351951e-07], [0.01443144400786929, 0.01443144400786929], [0.008563021458520875, 0.008563021458520564], [4.414876716249855e-05, 4.414876716249855e-05], [3.008792116623904e-05, 3.008792116623904e-05], [6.5826553719011345e-24, 1.921734728161301e-55], [9.080395292908562e-30, 0.014230945521164776], [9.085186027487801e-13, 0.010043572466719126], [8.93446239393228e-13, 8.93446239393228e-13], [0.00043565769764805076, 1.7112782386195592e-43], [0.00027645955662101805, 1.0155777807474605e-18], [5.936101037873387e-19, 5.936101037873387e-19], [0.014363482342783025, 0.014363482342783025], [0.009472250875252155, 0.009472250875252155], [1.4679827378792019e-12, 1.4679827378792019e-12], [2.1332218340365223e-21, 2.1332218340365223e-21], [1.7195817238118657e-21, 1.7195817238118657e-21], [3.770421448159074e-87, 3.770421448159074e-87], [0.0, 2.6300352039518566e-124], [0.0, 1.945929108316937e-124], [0.0, 0.0], [3.1280285467222e-05, 9.62981221806722e-07], [5.473248570246098e-15, 5.473248461435246e-15], [8.463740117661957e-62, 8.463740117661957e-62], [0.006325715976974673, 0.006325715976974672], [0.0040276567807475015, 0.0040276567807475015], [1.2881636430646304e-14, 1.2881116963140957e-14], [1.0673307868279985e-85, 1.963035761201318e-08], [2.9860630850644973e-37, 1.138603342328134e-08], [1.6780847767367784e-37, 1.6780847767367784e-37], [0.0, 0.0], [4.132384897072475e-26, 4.132384897072475e-26], [6.448697965109989e-235, -8.161730695551564e-131], [7.666571451074915e-06, 7.666571451074915e-06], [5.865559982851292e-06, 5.865559982851292e-06], [9.222218070767316e-27, 9.72828140082184e-62], [0.0, 7.229943003718587e-126], [0.0, 6.232825074307716e-126], [0.0, 0.0], [1.1858823885931575e-21, 1.1858823885931575e-21], [8.582535290688731e-22, 8.582535290688731e-22], [2.602730260563458e-90, 2.602730260563458e-90], [6.201778967284362e-105, 6.201778967284362e-105], [4.476632794212359e-105, 4.476632794212359e-105], [0.0, 0.0], [3.1543158181844134e-92, 3.1543158181844134e-92], [3.2050692336415605e-92, 3.2050692336415605e-92], [0.0, 0.0], [6.964311587891353e-24, 6.964311587891353e-24], [5.378834564068905e-24, 5.378834564068905e-24], [9.943898906074473e-99, 9.943898906074473e-99], [7.566589614000759e-24, 7.566589614000759e-24], [6.580503808099282e-24, 6.580503808099282e-24], [2.8576727102034907e-98, 2.8576727102034907e-98], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [3.933349506924847e-81, 3.517073330806163e-08], [2.0682083428093057e-35, 2.2226715041613716e-08], [1.3034120266814078e-35, 1.3034120266814078e-35], [0.009875841962490221, 7.680137231338715e-31], [0.007382372759528657, 3.2594692393395703e-13], [4.5881027966716856e-31, 2.6045987587884296e-13], [0.0, 0.0], [1.207756588241423e-48, 1.207756588241423e-48], [0.0, -8.933882740907378e-243], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [1.3560970612180759e-17, 4.2322777921828304e-07], [9.26888139843067e-18, 9.268881459590496e-18], [3.267727983959688e-73, 3.267727983959688e-73], [5.8897297968270875e-05, 5.8897297968270875e-05], [3.3937680992613405e-05, 3.3937680992613405e-05], [2.1066645437805166e-53, 2.1066645437805166e-53], [1.8773852169658594, 1.8773852169658594], [2.1086656124996384, 2.110876038651522], [1.8224585761940146e-06, 0.0090870361322893], [0.0, 0.0], [3.476962353452993e-80, 3.476962353452993e-80], [2.895909354006043e-80, 2.895909354006043e-80], [7.778151387901291e-78, 7.778151387901291e-78], [5.168653741786916e-78, 5.168653741786916e-78], [7.443642922406591e-28, 7.443642922406591e-28], [6.502990735647937e-28, 6.502990735647937e-28], [0.0, 0.0], [0.0, 0.0], [8.039431582546279e-31, 8.039431582546279e-31], [7.645746626086436e-31, 7.645746626086436e-31], [6.486214042837711e-11, 0.0], [7.735779702272068e-11, 0.0], [0.0, 1.3042484093591995e-17], [0.0, 1.4045786602526866e-17], [0.0, 0.0], [1.136520396588691e-308, 1.136520396588691e-308], [0.2196617352472789, 0.2196617352472789], [0.2576037030542659, 0.25760370305426583], [1.1195468386197252e-07, 1.1204790399318405e-07], [2.1107246513429473e-11, 2.9582217882077284], [7.85705853940174e-05, 3.8886304264673734], [0.0004083121186212366, 0.0004083121186212366], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.07642970787457065, 2.2289772801710737e-86], [0.14281200960870172, 8.14752765564618e-60], [3.704004309363778e-09, 3.508612606013964e-21], [0.130655194064907, 0.13065519407098974], [0.11019004823750454, 0.11019004823750454], [0.0, 0.0], [4.103480272194689e-80, 0.0], [2.5959267779219153e-80, 0.0], [0.0, 5.303274737541493e-78], [0.0, 3.603592691956024e-78], [3.7629004501235963e-28, 3.7629004501235963e-28], [2.8212445227495784e-28, 2.8212445227495784e-28], [0.0, 0.0], [0.0, 0.0], [3.8441199197082775e-31, 3.8441199197082775e-31], [2.8892463955167127e-31, 2.8892463955167127e-31], [0.0, 4.8405867075772086e-11], [0.0, 3.764711727986006e-11], [6.722514292939615e-19, 6.722514292939615e-19], [2.456389710024997e-18, 2.456389710024997e-18], [5.786598546911183e-63, 0.0], [3.800169375486789e-63, 0.0], [0.10102243769318152, 0.10102243784203283], [0.07588877255709252, 0.07588877255709242], [3.804467924442892e-31, 0.001968944552660258], [6.522209789186647e-14, 0.002216163704836974], [2.0552431183654135e-13, 1.1562187360688453e-13], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.005853864826657925, 0.005853864826657925], [0.004264691870850398, 0.004264691871244235], [2.3278083295016836e-05, 2.3278083295016836e-05], [1.7691668100502558e-05, 1.7691668100502558e-05], [0.0, 0.0], [3.0611580141313035e-80, 3.0611580141313035e-80], [2.1609299091239405e-80, 2.1609299091239405e-80], [6.205294380045213e-78, 6.205294380045213e-78], [3.784876547413491e-78, 3.784876547413491e-78], [4.5032963180001455e-28, 4.5032963180001455e-28], [3.582696413408747e-28, 3.582696413408747e-28], [0.0, 0.0], [0.0, 0.0], [4.617823483710225e-31, 4.617823483710225e-31], [3.6816863295017157e-31, 3.6816863295017157e-31], [5.520395617149193e-11, 5.520395617149193e-11], [4.536568509177532e-11, 4.536568509177532e-11], [0.0, 9.958382245604764e-18], [0.0, 7.884465352774662e-18], [4.3930049288358705e-63, 4.3930049288358705e-63], [3.054349498239676e-63, 3.054349498239676e-63], [0.724940182580337, 0.724940182580337], [0.6740082431757022, 0.6740082431757018], [0.002533781650263917, 0.002533781650263905], [0.0030758524809944084, 0.0030758524809944084], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [1.1122091729393346e-05, 1.1130235287785554e-05], [0.03329257729454152, 0.03331695387466785], [1.673646505583362, 1.673646505583362], [1.0785913023487106, 1.0785913023487106], [0.0, 0.0], [6.832441982674718e-80, 6.832441982674718e-80], [4.703361696948109e-80, 4.703361696948109e-80], [1.723378901792929e-77, 1.723378901792929e-77], [1.153542739233887e-77, 1.153542739233887e-77], [1.079294304801587e-27, 1.079294304801587e-27], [9.99141627674372e-28, 9.99141627674372e-28], [0.0, 7.152393393298461e-19], [0.0, 6.914871230992414e-19], [1.1266915567525309e-30, 1.1266915567525309e-30], [1.0385439100243668e-30, 1.0385439100243668e-30], [1.1293792562130513e-10, 1.1293792562130513e-10], [1.0974445829375629e-10, 1.0974445829375629e-10], [2.1847869258621768e-17, 2.1847869258621768e-17], [2.0355223730616307e-17, 2.0355223730616307e-17], [1.3795294845656475e-62, 1.3795294845656475e-62], [9.464946562080248e-63, 9.464946562080248e-63], [1.4892846192972007e-06, 1.5818937082402055e-65], [1.471223943228729e-06, 1.1181096478400762e-28], [1.1579625769834455e-28, 1.1579625769834455e-28], [8.945669283085905e-06, 8.945669283085905e-06], [1.0112396352731327e-05, 1.0112396352731327e-05], [3.0669030488728585e-25, 1.2501244670959188e-57], [4.94866467651495e-06, 4.94866467651495e-06], [5.7312877044789005e-06, 5.731287712636785e-06], [3.3602441887804406e-26, 3.3602441887804406e-26], [7.491545200297982e-76, 1.6052351995215682e-07], [4.765741202245132e-33, 1.2603866298333497e-07], [3.335843921222e-33, 3.335843921222e-33], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [4.7197467324205614e-12, 4.7197467324205614e-12], [3.992314698054613e-12, 3.992314698054613e-12], [4.318647881203248e-51, 4.318647881203248e-51], [0.0, 0.0], [0.0, 3.1656709748522016e-80], [3.242508749159196e-78, 3.242508749159196e-78], [3.166370261916703e-78, 3.166370261916703e-78], [6.887119054429011e-28, 6.887119054429011e-28], [6.009817895193255e-28, 6.009817895193255e-28], [0.0, 0.0], [0.0, 0.0], [7.479064857208126e-31, 7.479064857208126e-31], [6.374233607339908e-31, 6.374233607339908e-31], [6.433704942241596e-11, 6.433704942241596e-11], [6.260023220053783e-11, 6.260023220053783e-11], [1.1705717692507877e-17, 1.1705717709169732e-17], [1.1172254251961422e-17, 1.1172254251961422e-17], [3.2600735956534376e-63, 3.2600735956534376e-63], [2.992668848211483e-63, 2.992668848211483e-63], [-3.9928103524253035e-31, 0.06534563046881536], [-4.745606824685069e-19, 0.0734101178203516], [0.00794777692275384, 0.007947776922753771], [0.00738263966277449, 0.00738263966277449], [3.757331307190212e-13, 6.526994031502145e-13], [3.9432591850214034, 3.9432591850214034], [3.8713641899486766, 3.9046565983131156], [0.026000358203701862, 0.05546968698134928], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [2.3074320532679966, 2.3074320532655275], [1.2604873564739825, 1.260487355576897], [0.0, 0.0], [4.868198795288072e-80, 4.868198795288072e-80], [3.8913500686337335e-80, 3.891350065864228e-80], [3.4169809142901774e-305, 1.3784378387288471e-77], [0.0, 1.0262774725955e-77], [0.0, 1.1117065000900086e-27], [0.0, 1.1361735711324986e-27], [0.0, 0.0], [0.0, 0.0], [1.172990595043259e-30, 1.172990595043259e-30], [1.1831293492624432e-30, 1.1831293492624432e-30], [1.0454260712285427e-10, 0.0], [1.1783160642548189e-10, 0.0], [0.0, 1.9947723419074788e-17], [0.0, 2.176729242799549e-17], [9.799148088184798e-63, 9.799148088184798e-63], [7.661739984896577e-63, 7.661739984896577e-63], [0.11313078340043681, 0.11313078348095171], [0.15057668909599584, 0.15057668909599584], [1.8566604922188873e-08, 1.858439933441404e-08], [4.469626290672426e-05, 1.355431342480527e-52], [4.214196051237668e-05, 4.519330440175521e-23], [1.3114437092450357e-22, 8.623281422743543e-23], [3.0446947779999856, 3.0446947779999856], [3.190177498550418, 3.189977134469757], [0.0008890381586713484, 0.00032196034179765923], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.7011438914429836, -6.791995065271415e-17], [2.005496680235322, -3.899136656657293e-10], [0.0001975949587756644, 0.00025519080870093746], [0.0, 0.0], [3.148960603478193e-80, 3.148960603478193e-80], [2.3527781299498394e-80, 2.3527781299498394e-80], [0.0, 1.180583082947449e-77], [0.0, 7.040496298097311e-78], [5.435400013696669e-28, 5.435400013696669e-28], [4.60703028675939e-28, 4.60703028675939e-28], [2.596521513519699e-19, 0.0], [2.6747268615986665e-19, 0.0], [6.29385207330097e-31, 6.29385207330097e-31], [5.082436922319067e-31, 5.082436922319067e-31], [4.3922291187258775e-11, 4.3922291187258775e-11], [4.5005784423757006e-11, 4.5005784423757006e-11], [8.240546177672987e-18, 8.240546177672987e-18], [8.065035567998411e-18, 8.065035567998411e-18], [7.518785969756839e-63, 7.518785969756839e-63], [4.987332878420321e-63, 4.987332878420321e-63], [3.214145543554456e-17, 3.214145543554456e-17], [2.61836652357718e-17, 2.61836652357718e-17], [8.814974496780138e-71, 8.814974496780138e-71], [4.798363760787278e-09, 4.798363760787278e-09], [4.821076721961192e-09, 4.821076721961192e-09], [1.6999495172817348e-37, 6.51944825850298e-85], [4.144771549676906e-17, 4.144771549676906e-17], [4.336470884729599e-17, 4.336470884729599e-17], [1.3502113035977495e-69, 1.3502113035977495e-69], [8.736432178013366e-08, 8.736432178013366e-08], [5.488987706981254e-08, 5.488987706981254e-08], [5.611028508469896e-34, 5.611028505193623e-34], [2.3243096382962236e-157, 5.936483845289055e-17], [1.3978976339388619e-69, 5.090352106146293e-17], [1.2762369820484186e-69, 1.2762369820484186e-69], [3.1980803922541105e-25, 3.1980803922541105e-25], [2.1555386739909407e-25, 2.1555386739909407e-25], [1.626160093814358e-103, 1.626160093814358e-103], [0.0, 0.0], [1.7808753378302787e-80, 1.7808753378302787e-80], [1.690818900351939e-80, 1.690818900351939e-80], [0.0, 0.0], [1.4089812588714919e-78, 1.4089812588714919e-78], [6.857511617114956e-28, 6.857511617114956e-28], [5.999165017737127e-28, 5.999165017737127e-28], [3.3702889753434305e-19, 3.3702889753434305e-19], [3.694890147417098e-19, 3.694890147417098e-19], [7.116853262035351e-31, 7.116853262035351e-31], [7.288008157248944e-31, 7.288008157248944e-31], [6.670619156350795e-11, 6.670619156350795e-11], [6.436947785164433e-11, 6.436947785164433e-11], [0.0, 1.3186574326085312e-17], [0.0, 1.1947655936541419e-17], [1.4201663021975956e-63, 1.4201663021975956e-63], [2.0768565316226053e-63, 2.0768565316226053e-63], [2.959948537094547e-256, 6.114221205668089e-09], [3.2172060052492e-263, 4.379641635957006e-09], [4.704e-321, 3.213388549388354e-89], [3.232027999435182e-06, 3.232027999435182e-06], [2.7186673758940897e-06, 2.7186673758940897e-06], [1.0860577149651557e-27, 1.0860577149651557e-27], [0.00020190619159480145, 0.00020190619159480145], [0.0001583930996726404, 0.0001583930996726404], [1.2451162429465355e-20, 1.2451162429465355e-20], [6.364333241285456e-13, 4.032037052999227e-119], [8.893548799564676e-13, 3.9082824250693044e-53], [1.3064499787262336e-52, 1.3064499787262336e-52], [0.001637443341923045, 0.0], [0.001375545415602204, 0.0], [1.100279355499872e-16, 0.0], [9.787391170766135e-07, 9.787391170766135e-07], [6.884984629096781e-07, 6.884984629096781e-07], [1.1649638922574828e-30, 1.1649638922574828e-30], [0.0, 0.0], [1.8300432059940774e-80, 1.8300432059940774e-80], [8.4175674324731785e-81, 8.4175674324731785e-81], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [-1.4402300717747026e-127, -1.8521846558699191e-177], [2.3202091189754133e-63, 2.3202091189754133e-63], [0.0012370983133204768, 1.7983887490797187e-30], [0.0002266899695313755, 9.496331475520483e-13], [2.8481633793220016e-14, 2.8481633793220016e-14], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.005752154785843018, 0.005752154785843018], [0.0, 0.0], [3.635406696933735e-13, 3.635406696933735e-13], [0.0, 0.0], [0.0, 0.0], [0.0, 0.0], [0.0021244827850973717, 0.0021244827850973717], [0.002316383246508824, 0.002316383246508824], [4.32668725179485e-15, 4.326388330762671e-15], [2.5172203823654903e-06, 2.5172203823654903e-06], [1.5200339096960418e-27, 1.5200339096960418e-27], [3.013877708207437e-27, 3.0138777082074274e-27], [0.0, 0.0]]
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge/RepeatedKFold/best_worst_per_point.png'
metrics_dict = {'R2': (True, <function r2_score at 0x7fefaa5992f0>), 'root_mean_squared_error': (False, <function root_mean_squared_error at 0x7fefaa599378>), 'mean_absolute_error': (False, <function mean_absolute_error at 0x7fefac1236a8>), 'rmse_over_stdev': (False, <function rmse_over_stdev at 0x7fefaa599400>)}
avg_stats = OrderedDict([('R2', (0.012328956771457334, 0.007218821590948318)), ('root_mean_squared_error', (141.57244937539684, 5.714592792315468)), ('mean_absolute_error', (105.13763442638033, 2.7496719311951106)), ('rmse_over_stdev', (1.4855310198404497, 0.08361794873500152))])
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_best_worst_per_point(y_true, y_pred_list, savepath, metrics_dict, avg_stats, label)
display(Image(filename='best_worst_per_point.png'))
